## This script

Generates a Pig script that cleans up a CSV containing some numerical and timestamp data.


### Storing as Parquet

[Cloudera Blog](http://blog.cloudera.com/blog/2014/03/how-to-use-parquet-with-impala-hive-pig-mapreduce/)

`grunt> store A into '/output_dir' USING parquet.pig.ParquetStorer;`

### Thrift for Pig

```
if [ -e /opt/cloudera/parcels/CDH ] ; then
  CDH_BASE=/opt/cloudera/parcels/CDH
else
  CDH_BASE=/usr
fi
THRIFTJAR=`ls -l $CDH_BASE/lib/hive/lib/libthrift*jar | awk '{print $9}' | head -1`
export PIG_OPTS="-Dpig.additional.jars=$THRIFTJAR"
```

In [82]:
import sys, json, re

In [83]:
script = r"""A = LOAD '{src}' using PigStorage('\n') as (line:chararray);
B = FOREACH A GENERATE FLATTEN(STRSPLIT(line, '{delim}')) {load_as_schema};
X = FOREACH B GENERATE {fields};
STORE X INTO '{dest}' USING {store_format};"""

In [84]:
def gen_load_schema(cols, numeric_cols=None):
    if numeric_cols and any(x > cols for x in numeric_cols):
        print("gen_load_schema: Incorrect columns sizes")
        return;
    
    schema = []
    for i in range(cols):
        _type = "chararray"
        if numeric_cols and i+1 in numeric_cols:
            _type = "bigdecimal"
        schema.append("f{}:{}".format(i, _type if i != 0 else "chararray"))
    return schema

In [85]:
def gen_fields(cols, date_cols=None, numeric_cols=None, date_fmt="yyyy-MM-dd HH:mm:ss"):
    if date_cols and any(x > cols for x in date_cols):
        print("gen_fields: Incorrect columns sizes")
        return;
    if numeric_cols and any(x > cols for x in numeric_cols):
        print("gen_fields: Incorrect columns sizes")
        return;
    
    fields = []
    for i in range(cols):
        field = "$" + str(i)
        if i == 0: 
            field = "LTRIM({})".format(field)
            if numeric_cols and i+1 in numeric_cols: # want to cast after TRIM
                field = "(bigdecimal) "+ field
        if date_cols and i+1 in date_cols:
            ''' Thanksgiving data = '20-NOV-16'... Need month as 'Nov' '''
#             field = "CONCAT(SUBSTRING({0},0,3), UCFIRST(SUBSTRING({0},3,9)))".format(field)
#             field = "ToString(ToDate({},'{}', 'UTC'), 'yyyy-MM-dd HH:mm:ss')".format(field, date_fmt)
            
            field = "ToDate({}, '{}', 'UTC')".format(field, date_fmt)
            field = "ToUnixTime({})".format(field)
        fields.append(field)
    return fields

In [86]:
def gen_regex_capture(cols, with_sep=None, sep=','):
    if with_sep and any(x > cols for x in with_sep):
        print("gen_regex_capture: Incorrect columns sizes")
        return;
    capture_no_sep = '([^{}]*)'.format(sep)
    capture_with_sep = '(.*?)'
    captures = [capture_with_sep if i + 1 in with_sep else capture_no_sep for i in range(cols)]
    return sep.join(captures) + '$'

In [142]:
filename = "conf_admin_prod_sku_dm.json"
with open('/Users/moorejm/workspace/clients/michaels/conf/' + filename) as f:
    config = json.load(f)

In [143]:
# if len(sys.argv[1:]) < 2:
#     print("Usage:" + sys.argv[0] + " total_columns date_columns")
# cols = sys.argv[1]
# dates = sys.argv[2].split(",")

root_input = config['master_data']
data = config['data']
field_data = data['fields']

# Generate fields
num_cols = field_data['amount']
date_cols = field_data['date_columns']
numeric_cols = field_data['numeric_columns']
date_fmt = "yyyy-MM-dd HH:mm:ss" if 'date_fmt' not in field_data else field_data['date_fmt']
schema = gen_load_schema(num_cols, numeric_cols)
fields = gen_fields(num_cols, date_cols, numeric_cols, date_fmt)

# B
script_data = {
    "src" : "{}{}".format(root_input['src'], data['src']),
    "dest": "{}{}".format(root_input['dest'], data['dest']),
    "delim": data['delim'],
    "load_as_schema": "AS ({})".format(", ".join(schema)),
    "fields" : ", ".join(fields),
    "store_format": config['output']['store_format'] # PigStorage(',') or parquet.pig.ParquetStorer
}

# SET parquet.compression gzip; -- The default compression is snappy
    
print(script.format(**script_data))


A = LOAD 'hdfs:///user/ec2-user/prod_sku_dm.txt' using PigStorage('\n') as (line:chararray);
B = FOREACH A GENERATE FLATTEN(STRSPLIT(line, ',\\s*')) AS (f0:chararray, f1:chararray, f2:chararray, f3:chararray, f4:chararray, f5:chararray, f6:chararray, f7:chararray, f8:chararray, f9:chararray, f10:chararray, f11:chararray, f12:chararray, f13:chararray, f14:chararray, f15:chararray, f16:chararray, f17:chararray, f18:chararray, f19:chararray, f20:chararray, f21:chararray, f22:chararray, f23:chararray, f24:chararray, f25:chararray, f26:chararray, f27:chararray, f28:chararray, f29:chararray, f30:chararray, f31:chararray, f32:chararray, f33:chararray, f34:chararray, f35:chararray, f36:chararray, f37:chararray, f38:chararray, f39:chararray, f40:chararray, f41:chararray, f42:chararray, f43:chararray, f44:chararray, f45:chararray, f46:chararray, f47:chararray, f48:chararray, f49:chararray, f50:chararray, f51:chararray, f52:chararray, f53:chararray, f54:chararray, f55:chararray, f56:chararray, f5

In [10]:
expect = '([^,]+),([^,]+),([^,]+),([^,]+),(.+?),([^,]+),([^,]+),([^,]+),([^,]+),([^,]+)$'

gen_regex_capture(10, with_sep=[5]) == expect

True

In [18]:
samp = """764,1585,S,156,10,1,1,1,2,2001-04-11 00:00:00,Y,2016-05-03 00:00:00,1585 ANDERSON, SC,1585,AND,Store,802,0802 WELSH,8,0008 MORALES - MID ATLANT,1,All Areas,1,All Chains,1,Michaels Stores Inc.,US dollar,NORTH HILLS COMMONS, PHASE II 3521 CLEMSON BLVD,ANDERSON,SC,South Carolina,US,USA,29621,COURTNEY S HOLMAN,1,All Store Format,19353,23923,1,All Promo Zones,,0,,,,N,,7,,,,C,,,,,,2001-06-21 00:00:00,,4444-04-04 00:00:00,97,57,L3,GREENVILLE, SC,2002-07-07 00:00:00,,open"""
# for i,x in enumerate(samp.split(',')):
#     print(i, x)

# import re
with_sep = [13,65]
gen_regex_capture(68, with_sep=with_sep)

'([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),(.*?),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),([^,]*),(.*?),([^,]*),([^,]*),([^,]*)$'

In [146]:
regex = r"^(-?\d+),(-?\d+),(-?\d+),(-?\d+),(-?\d+),(-?\d+),(-?\d+),(-?\d+),(-?\d+),([^,]{1,2}),(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}),([^,]),(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}),([^,]{0,10}),([^,]{0,4}),(.{0,30}),([^,]{0,4}),(.{0,30}),([^,]{0,4}),(.{0,30}?),([^,]{0,4}),(.{0,30}?),([^,]{0,4}),(.{0,30}?),([^,]{0,4}),(.{0,30}?),([^,]{0,4}),(.{0,30}?),([^,]{0,6}),([^,]{0,30}),([^,]{0,6}),(.{0,30}),([^,]{0,10}),([^,]{0,30}?),(.{0,100}),(.{0,30}?),(.{0,30}?),(-?\d*),([^,]{0,4}),(.{0,20}),(|\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}),([^,]{0,2}),([^,]{0,30}),([^,]{0,2}),([^,]{0,30}),([^,]{0,2}),([^,]{0,30}),([^,]{0,30}),([^,]{0,30}),([^,]{0,30}),([^,]{0,30}),([^,]{0,30}),([^,]{0,30}),(-?\d*),([^,]?),([.\d]{0,20}),([.\d]{0,20}),([^,]?),([.\d]{0,20}),(|\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}),(-?\d*),(-?\d*),([^,]{0,10}),(.{0,30}),(.{0,30}),([^,]?),(-?\d*),(-?\d*),([^,]?),(-?\d*),(-?\d*),(-?\d*)$"
sizes = set()
top = 50000

with open("/Users/moorejm/Downloads/prod_sku_dm.txt") as f:
    expect_cols = 72
    for line in f:
        if top <= 0:
            break
        line = line.rstrip()
        splits = line.split(',')
        cols = len(splits)
#         sizes.add(cols)
#         matches = re.finditer(regex, line)
#         for i,m in enumerate(matches):
#             groups = m.groups()
#             sizes.add(len(groups))
#             if len(groups) != expect_cols:
#                 print("nope")
        if cols >= 77:
            print(line)
        top -= 1
sizes

738803,14777,1652,121,73,12,1,-1,-1,S,2012-05-18 00:00:00,Y,2017-01-16 00:00:00,10249167,9256,9256 TOWER BASKET SBA,114,114 PERFECT PICK-UPS,75,75 IMPULSE,11,IMPULSE,6,CELEBRATIONS,1,Michaels Stores Inc.,,,,,,,,,10249167 PAPER FLOWER LARGE FAB FINDS,PAPER FLOWER LARGE F,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,0.4200,0.5000,Y,0.6244,2015-10-04 00:00:00,3,7745,84325,LY8265-2,5,6,8,9,10,LI & FUNG - LIAOYANG ZHENXING ,Y,180,30,N,7,,
721133,15212,1652,121,73,12,1,-1,-1,S,2012-01-24 00:00:00,N,2017-01-16 00:00:00,10232594,9797,9797 Q4 TOWER BASKETS,114,114 PERFECT PICK-UPS,75,75 IMPULSE,11,IMPULSE,6,CELEBRATIONS,1,Michaels Stores Inc.,,,,,,,,,10232594 PAPER MACHE WINTR COTTAGE Q412,PAPER MACHE WINTR CO,Staple Sku,,CA,Case,2016-07-23 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,0.4500,0.0100,Y,0.5453,2012-12-25 00:00:00,3,7745,84325,LY8637-2,3,5,8,9,10,LI & FUNG - LIAOYANG ZHENXING ,Y,1056,66,N,7,,


set()

In [45]:
samp = """322997,11826,1716,122,78,1,1,-1,-1,S,2005-01-25 00:00:00,Y,2016-04-15 00:00:00,10375394,3445,3445 KNITTING/CROCHET,15,015 TEXTILE CRAFTS,76,76 BOOKS,6,BOOKS & MAGAZINES,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10375394 BK VICTORIAN CHARM,BK VICTORIAN CHA,Staple Sku,,EA,Each,4444-04-04 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,N,2.2900,1.4900,Y,2.6771,2010-06-22 00:00:00,3,7134,75523,YR25949,HDA INC,Y,1,1,,,,
288437,15721,1945,100,44,3,1,-1,-1,S,2002-11-29 00:00:00,N,2016-09-01 00:00:00,10335330,9998,9998 DISCONTINUED-NON PI,997,997 DISCONTINUED,44,44 STICKERS & EMBELLISH,8,SCRAPBOOKING,3,PAPERCRAFTING,1,Michaels Stores Inc.,,,,,,,,,10335330 DC LG COWBOY HAT,DC LG COWBOY HAT,Staple Sku,,CA,Case,2015-06-20 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,N,0.1200,0.0100,Y,0.1403,2005-01-16 00:00:00,3,4232,34155,22501,MAKING MEMORIES INC,N,24,24,,,,
322517,16553,1837,103,43,2,1,-1,-1,S,2005-01-11 00:00:00,N,2016-02-26 00:00:00,10374643,684,0684 CMAS CRAFT BASICS DA,49,049 SEASONAL,51,51 KIDS CRAFTS,7,KIDS CRAFTS,2,KIDS/TWEENS,1,Michaels Stores Inc.,,,,,,,,,10374643 GLITTER GLUE PENS MINI BRIGHT,GLITTER GLUE PEN,Staple Sku,,CA,Case,2015-08-15 00:00:00,,,,,,,Seasonal,Yes,Direct,Discontinued,Buyer,No,0,N,0.4500,0.0100,Y,0.5261,2005-12-23 00:00:00,3,4571,362,9296 01,FIBRE-CRAFT LLC,Y,6,6,,,,
322037,13330,1616,96,3,1,1,-1,-1,S,2004-12-29 00:00:00,Y,2016-02-26 00:00:00,10373960,7470,7470 SIMPLY SOFT-CARON,106,106 BASIC ACRYLIC/ACRYLI,40,40 YARN,3,NEEDLE CRAFTS,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10373960 YRN SIMPLY SFT BRT MANGO,YRN SIMPLY SFT BRT,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,1.7700,0.0100,Y,2.0916,2009-04-05 00:00:00,1,353,130,H990B9502,SPINRITE CORP..,Y,80,4,N,7,7,
321557,13885,1841,107,60,2,1,-1,-1,S,2004-12-15 00:00:00,Y,2016-07-26 00:00:00,10373238,8556,8556 BRUSH ON PAINT,50,050 PAINTS & DYE,56,56 APPAREL CRAFTS,4,APPAREL CRAFTS,2,KIDS/TWEENS,1,Michaels Stores Inc.,,,,,,,,,10373238 PAINT CRIMSON RED 4OZ TU,PAINT CRIMSON RE,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,2.3100,0.5900,Y,2.7297,2015-03-08 00:00:00,3,6022,716,20405,DUNCAN  ENTERPRISES,Y,36,3,N,4,4,
321077,17972,2137,100,44,3,1,-1,-1,S,2004-12-07 00:00:00,N,2016-09-01 00:00:00,10372663,2162,2162 FLOWERS-SBA,525,525 FLOWERS (STICKERS),44,44 STICKERS & EMBELLISH,8,SCRAPBOOKING,3,PAPERCRAFTING,1,Michaels Stores Inc.,,,,,,,,,10372663 HRTS STRS  LIF JRNY CLRLY  YRS,HRTS STRS  LIF J,Staple Sku,,CA,Case,2015-08-15 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,0.9963,0.2500,Y,1.1647,2006-02-27 00:00:00,3,4569,36191,280083,K AND COMPANY,Y,6,6,,,,
813085,24332,2176,121,73,12,1,-1,-1,S,2014-04-25 00:00:00,Y,2017-01-16 00:00:00,10372895,8325,8325 GIFTS FOR HER,761,761 GIFTS & EVENTS,75,75 IMPULSE,11,IMPULSE,6,CELEBRATIONS,1,Michaels Stores Inc.,,,,,,,,,10372895 3PC COSMETIC TRAVEL BAG,3PC COSMETIC TRAVEL ,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Seasonal,Yes,Warehouse,Discontinued,Buyer,No,0,N,3.9200,1.9900,Y,4.8068,2015-03-11 00:00:00,3,7118,75169,MI-623  AST,ENCHANTE ACCESSORIES (IMPORT),Y,4,4,,,,
77474,15291,1976,97,3,1,1,-1,-1,S,1990-01-01 00:00:00,N,2016-02-26 00:00:00,10090348,9999,9999 SUBCLASS NOT IN USE,998,998 CLASS NOT IN USE,41,41 STITCHERY,3,NEEDLE CRAFTS,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10090348 PET. GLASS BEAD GINGER,PET. GLASS BEAD,Staple Sku,,CA,Case,2015-04-11 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,0.4200,0.0100,Y,0.4578,1996-04-28 00:00:00,3,1455,20618,CLEARANCE,WICHELT IMPORTS INC,Y,3,3,,,,
811645,25425,2469,139,26,1,1,-1,-1,S,2014-04-09 00:00:00,Y,2016-04-13 00:00:00,10370712,9416,9416 STORAGE TEST,807,807 TESTS,46,46 BEADS & FINDINGS,1,JEWELRY & BEADS,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10370712 LANYARD - PURPLE PAISLEY,LANYARD - PURPLE PAI,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Seasonal,Yes,Warehouse,Discontinued,Buyer,No,0,N,1.5000,0.9900,Y,1.7725,2015-07-12 00:00:00,1,8674,97493,EVM 9421-5,EVERYTHING MARY LLC (DOMESTIC),Y,24,3,,,,
812125,25093,1604,96,3,1,1,-1,-1,S,2014-04-14 00:00:00,Y,2016-04-19 00:00:00,10371504,9075,9075 DISCONTINUED FASHION,103,103 FASHION YARN,40,40 YARN,3,NEEDLE CRAFTS,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10371504 PREMIER HOME TM RAFFIA AZURITE,PREMIER HOME TM RAFF,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Seasonal,Yes,Warehouse,Discontinued,Buyer,No,0,N,1.6500,2.4900,Y,1.9498,2015-07-12 00:00:00,3,8392,92064,99-01,PREMIER YARNS COMPANY,Y,24,3,,,,
319637,17710,2122,103,43,2,1,-1,-1,S,2004-10-07 00:00:00,N,2016-02-26 00:00:00,10370217,1905,1905 FOAM ACCESSORIES,361,361 FOAM,51,51 KIDS CRAFTS,7,KIDS CRAFTS,2,KIDS/TWEENS,1,Michaels Stores Inc.,,,,,,,,,10370217 FOAMIE DOOR HANGER RECT BLACK,FOAMIE DOOR HANG,Staple Sku,,CA,Case,2015-08-15 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,0.2500,0.0100,Y,0.2923,2007-04-20 00:00:00,3,330,128,1046-15,DARICE INC,Y,6,6,,,,
319157,19967,2233,97,3,1,1,-1,-1,S,2004-09-23 00:00:00,N,2016-02-26 00:00:00,10369519,4081,4081 COTTON CRAFT FABRIC,602,602 FABRIC,41,41 STITCHERY,3,NEEDLE CRAFTS,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10369519 FAB-LARGE PRINT 10,FAB-LARGE PRINT,Staple Sku,,CA,Case,2015-08-15 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,2.9900,0.0100,Y,3.4954,2006-02-20 00:00:00,3,6487,59887,MK-LGPRINT-10,FABRIC EDITIONS LLC,Y,3,3,,,,
318677,15262,1947,104,26,1,1,-1,-1,S,2004-09-10 00:00:00,N,2015-08-08 00:00:00,10368844,9999,9999 SUBCLASS NOT IN USE,997,997 DISCONTINUED,52,52 JEWELRY,1,JEWELRY & BEADS,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10368844 BEAD GLASS RAINBOW MIX TURQ AB,BEAD GLASS RAINB,Staple Sku,,CA,Case,2015-08-08 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,0.7510,0.0100,Y,0.8875,2008-02-11 00:00:00,3,1854,21252,2576605,COUSIN CORPORATION OF AMERICA,Y,240,3,N,7,7,
811165,22941,1604,96,3,1,1,-1,-1,S,2014-04-04 00:00:00,Y,2016-02-26 00:00:00,10369636,6940,6940 L&T CARMELLA,103,103 FASHION YARN,40,40 YARN,3,NEEDLE CRAFTS,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10369636 CARMELLA AUTUMN,CARMELLA AUTUMN,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,1.3000,2.4900,Y,1.6984,2015-05-24 00:00:00,3,8168,88412,121024-2,LI & FUNG - GLITTER GROUP CO L,Y,126,3,N,2,,
318197,17710,2122,103,43,2,1,-1,-1,S,2004-08-25 00:00:00,N,2016-02-26 00:00:00,10368169,1905,1905 FOAM ACCESSORIES,361,361 FOAM,51,51 KIDS CRAFTS,7,KIDS CRAFTS,2,KIDS/TWEENS,1,Michaels Stores Inc.,,,,,,,,,10368169 FOAM KIT DR HNGR PRINCESS 4PC,FOAM KIT DR HNGR,Staple Sku,,CA,Case,2015-08-08 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,1.3500,0.0100,Y,1.5782,2006-01-23 00:00:00,3,330,128,1045-06,DARICE INC,Y,3,3,,,,
317717,19090,1814,122,78,1,1,-1,-1,S,2004-08-10 00:00:00,Y,2016-04-15 00:00:00,10367382,3236,3236 SCRAPBOOK,35,035 PAPER CRAFTING,76,76 BOOKS,6,BOOKS & MAGAZINES,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10367382 BK-COLLAGE CREATION,BK-COLLAGE CREAT,Staple Sku,,EA,Each,4444-04-04 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,N,11.5900,19.9900,Y,13.5490,2012-01-21 00:00:00,3,8016,86588,9781581805468,BAKER AND TAYLOR INC,Y,1,1,,,,
293717,25801,2443,116,82,4,1,-1,-1,S,2003-03-11 00:00:00,N,2016-04-30 00:00:00,10341098,9806,9806 BOB ROSS BRUSHES,781,781 OIL BRUSHES,70,70 PAINTING ART SUPPLIE,16,ART SUPPLIES,4,FINE ART & FRAMES,1,Michaels Stores Inc.,,,,,,,,,10341098 BRSH ROSS BRISTLE B WILDLIFE,BRSH ROSS BRISTL,Staple Sku,,EA,Each,2016-04-30 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,7.9400,0.0100,Y,8.6546,2010-05-30 00:00:00,3,1453,20616,WAR6345,MACPHERSON'S,Y,1,1,,,,
296117,20405,2270,99,44,3,1,-1,-1,S,2003-03-31 00:00:00,N,2016-05-06 00:00:00,10343626,4495,4495 COLOR IT,761,761 SOCIAL STATIONARY,43,43 PAPER, CARDS & ALBUM,8,SCRAPBOOKING,3,PAPERCRAFTING,1,Michaels Stores Inc.,,,,,,,,,10343626 CARD GIBSON M-DAY COUNTER CRDS,CARD GIBSON M-DA,Staple Sku,,EA,Each,2015-07-04 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,N,0.6450,0.0100,Y,0.7031,2004-06-22 00:00:00,1,470,1461,64002,GIBSON GREETINGS,Y,1,1,,,,
316277,20069,2256,119,68,13,1,-1,-1,S,2004-06-15 00:00:00,N,2017-01-16 00:00:00,10365023,4170,4170 NFL,683,683 LICENSED SPORTS,73,73 REGIONAL MERCHANDISI,11,IMPULSE,10,SEASONAL,1,Michaels Stores Inc.,,,,,,,,,10365023 GRDN FLAG NEW ORLEANS SAINTS,GRDN FLAG NEW OR,Staple Sku,,CA,Case,2015-08-08 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,N,4.0000,5.0000,Y,4.6761,2005-06-20 00:00:00,3,230,1192,08375021,WINCRAFT INC,Y,2,2,,,,
282677,543,356,94,47,9,1,-1,-1,S,2002-08-23 00:00:00,N,2015-06-13 00:00:00,10328992,1531,1531 PRINTS,54,054 ARTWORK,33,33 CUSTOM FRAMING,15,15 CUSTOM FRAMING,8,SERVICES,1,Michaels Stores Inc.,,,,,,,,,10328992 PRNT TEMPLE UNIVERSITY,PRNT TEMPLE UNIV,Staple Sku,,EA,Each,2015-06-13 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,N,23.8000,4.9900,Y,27.8228,2007-03-07 00:00:00,3,5937,54747,PA992,CAMPUS IMAGES,N,1,1,,,,
315797,19967,2233,97,3,1,1,-1,-1,S,2004-05-24 00:00:00,N,2016-02-26 00:00:00,10364523,4081,4081 COTTON CRAFT FABRIC,602,602 FABRIC,41,41 STITCHERY,3,NEEDLE CRAFTS,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10364523 FAB-CRAFT FABRIC 46,FAB-CRAFT FABRIC,Staple Sku,,CA,Case,2015-08-08 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,1.2600,0.0100,Y,1.4730,2007-02-19 00:00:00,3,6487,59887,MK-1046,FABRIC EDITIONS LLC,Y,3,3,,,,
315317,16523,2047,99,44,3,1,-1,-1,S,2004-05-10 00:00:00,N,2016-05-06 00:00:00,10364035,653,0653 BOX CHRISTMAS CARDS,106,106 COLLECTIONS,43,43 PAPER, CARDS & ALBUM,8,SCRAPBOOKING,3,PAPERCRAFTING,1,Michaels Stores Inc.,,,,,,,,,10364035 CARD CORE LOW BX ASST UXBC-4L,CARD CORE LOW BX,Staple Sku,,CA,Case,2015-08-08 00:00:00,,,,,,,Seasonal,Yes,Direct,Discontinued,Buyer,No,0,N,3.9008,0.0100,Y,4.5601,2010-02-27 00:00:00,3,470,1461,3498318,GIBSON GREETINGS,Y,36,36,,,,
808285,22860,1893,118,74,1,1,-1,-1,S,2014-03-06 00:00:00,Y,2016-02-26 00:00:00,10364862,6858,6858 SC JOHNSON,77,077 SPRAY PAINT,72,72 DECORATIVE ART,2,PAINT & DECO ART,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10364862 CARPET CLRT INDIGO 10OZ,CARPET CLRT INDIGO 1,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,7.5000,14.9900,N,8.8627,,1,8668,97399,660088,S C JOHNSON AND SON,Y,6,6,N,1,,
314837,26016,2139,100,44,3,1,-1,-1,S,2004-04-26 00:00:00,Y,2016-09-01 00:00:00,10363519,119,0119 SPORTS-SBA,527,527 THEMED STICKERS,44,44 STICKERS & EMBELLISH,8,PAPERCRAFTING,3,PAPERCRAFTING,1,Michaels Stores Inc.,,,,,,,,,10363519 STK BASEBALL REG 3 SHT,STK BASEBALL REG,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,0.7760,0.1900,Y,0.9170,2010-02-21 00:00:00,3,1296,20065,06473,MRS GROSSMAN PAPER CO,Y,510,6,N,4,4,
314357,24437,2149,100,44,3,1,-1,-1,S,2004-04-19 00:00:00,N,2016-09-01 00:00:00,10363022,8432,8432 CHARACTERS-SBA,537,537 TEACHER & KIDS (STIC,44,44 STICKERS & EMBELLISH,8,SCRAPBOOKING,3,PAPERCRAFTING,1,Michaels Stores Inc.,,,,,,,,,10363022 STKR EPOXY CIRCLES,STKR EPOXY CIRCL,Staple Sku,,CA,Case,2015-08-08 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,1.3151,0.0100,Y,1.5374,2006-07-03 00:00:00,3,8218,88940,SPTTYEP01,EK SUCCESS LTD (STICKO PRODUCT,Y,6,6,,,,
809245,17742,1740,118,74,1,1,-1,-1,P,2014-03-13 00:00:00,Y,2016-04-18 00:00:00,10366275,1935,1935 SUBCLASS NOT IN USE,20,020 CRAFT PAINT,72,72 DECORATIVE ART,2,PAINT & DECO ART,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10366275 DISPLAYER,DISPLAYER,Displayer Pack,,OZ,,4444-04-04 00:00:00,N,,N,,Y,,Seasonal,Yes,Warehouse,Active,Buyer,No,0,N,10.4800,,N,10.4800,,2,5135,42816,532P,DESIGN MASTER COLOR TOOL CDN,Y,1,1,,,,
298517,18355,2062,131,43,2,1,-1,-1,S,2003-05-05 00:00:00,N,2016-02-26 00:00:00,10346171,2515,2515 MELISSA AND DOUG,34,034 KIDS ACTIVITIES,58,58 KIDS ACTIVITY/HOBBY,7,KIDS CRAFTS,2,KIDS/TWEENS,1,Michaels Stores Inc.,,,,,,,,,10346171 TOOL KIT,TOOL KIT,Staple Sku,,CA,Case,2015-07-04 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,3.9900,5.0000,Y,4.6644,2005-07-25 00:00:00,3,6592,61485,826,MELISSA AND DOUG,Y,6,6,,,,
313397,15316,1749,85,51,8,1,-1,-1,S,2004-03-30 00:00:00,N,2017-01-16 00:00:00,10362014,9831,9831 $4.99 CMAS RBN,214,214 MESH/SPECIALTY RIBBO,24,24 RIBBON,10,RIBBON &  WEDDING,7,FLORAL/WEDDING,1,Michaels Stores Inc.,,,,,,,,,10362014 SPCL TRM NVY STN W/BEAD EDG,SPCL TRM NVY STN,Staple Sku,,CA,Case,2015-08-08 00:00:00,,,,,,,Seasonal,Yes,Direct,Discontinued,Buyer,No,0,N,1.0900,0.0100,Y,1.2742,2010-03-12 00:00:00,3,257,1215,HA4-13756-19,HIRSCHBERG SCHUTZ AND CO,Y,3,3,,,,
701243,26759,2509,85,51,12,1,-1,-1,S,2011-08-04 00:00:00,Y,2017-01-16 00:00:00,10211094,772,0772 CI 360-BLUE,858,858 PURPLE LABEL EVERYDA,24,24 RIBBON,10,RIBBON &  WEDDING,6,CELEBRATIONS,1,Michaels Stores Inc.,,,,,,,,,10211094 RBN 1" G TS BUZZ WOODY 3YDS,RBN 1" G TS BUZZ WOO,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,1.4900,0.4900,Y,1.7607,2014-11-23 00:00:00,3,812,17430,2261211560,WM WRIGHT CO,Y,72,3,N,1,,
159795,26466,2497,105,77,12,1,-1,-1,S,1990-01-01 00:00:00,Y,2016-06-15 00:00:00,10209863,492,0492 SUGAR SPARKLING,846,846 DECORATIONS & EMBELL,54,54 FOOD CRAFTING,12,FOOD CRAFT,6,CELEBRATIONS,1,Michaels Stores Inc.,,,,,,,,,10209863 FD-CA SPARKLES ORANGE .25OZ,FD-CA SPARKLES ORANG,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,1.6500,0.0100,Y,1.9289,2014-03-23 00:00:00,3,1338,20158,703-1308,WILTON INDUSTRIES (DOMESTIC),Y,2,2,,,,
99074,26328,2421,105,77,12,1,-1,-1,S,1990-01-01 00:00:00,Y,2016-10-27 00:00:00,10123238,354,0354 FOOD, ICING & COLORS,696,696 MIXES, MELTS & FROST,54,54 FOOD CRAFTING,12,FOOD CRAFT,6,CELEBRATIONS,1,Michaels Stores Inc.,,,,,,,,,10123238 FD-12 ICING COLOR KIT,FD-12 ICING COLOR KI,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Direct,Active,Buyer,No,1238,Y,5.1200,14.9900,N,5.9854,,3,1338,20158,601-5580,WILTON INDUSTRIES (DOMESTIC),Y,3,3,,,,
312917,16557,1837,103,43,2,1,-1,-1,S,2004-03-19 00:00:00,N,2016-02-26 00:00:00,10361516,688,0688 CMAS $1 STICKER BOOK,49,049 SEASONAL,51,51 KIDS CRAFTS,7,KIDS CRAFTS,2,KIDS/TWEENS,1,Michaels Stores Inc.,,,,,,,,,10361516 KIT ORN HOLIDAY MINIATURE,KIT ORN HOLIDAY,Staple Sku,,CA,Case,2015-08-01 00:00:00,,,,,,,Seasonal,Yes,Direct,Discontinued,Buyer,No,0,N,2.1000,0.0100,Y,2.4550,2004-11-17 00:00:00,3,1089,1916,204-47,MAC ENTERPRISES,Y,3,3,,,,
312437,12586,1638,120,59,13,1,-1,-1,S,2004-03-11 00:00:00,N,2016-03-19 00:00:00,10361017,6002,6002 BASIC FASHION CANDLE,111,111 FASHION PILLARS,74,74 CANDLES,21,CANDLES,10,SEASONAL,1,Michaels Stores Inc.,,,,,,,,,10361017 CNDL 4X6 GREEN GARDENIA TXTRD,CNDL 4X6 GREEN G,Staple Sku,,CA,Case,2016-03-19 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,4.2200,0.0100,Y,4.9868,2008-11-23 00:00:00,3,6639,63628,84010406203,DOMISTYLE CANDLES,Y,4,4,N,1,1,
311957,13276,1604,96,3,1,1,-1,-1,S,2004-03-05 00:00:00,Y,2016-02-26 00:00:00,10360518,7416,7416 LANDSCAPES-LIONBRAND,103,103 FASHION YARN,40,40 YARN,3,NEEDLE CRAFTS,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10360518 YN LANDSCAPES DEEP SEA,YN LANDSCAPES DE,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,3.0000,1.0000,Y,3.5451,2006-11-22 00:00:00,1,928,180,540-279,LION BRAND YARN,Y,180,3,N,8,8,
807325,11965,1791,106,43,2,1,-1,-1,S,2014-02-25 00:00:00,Y,2016-07-16 00:00:00,10362191,3911,3911 PAINT,30,030 KIDS ART MATERIALS,55,55 KIDS ART & EDUCATION,7,KIDS CRAFTS,2,KIDS/TWEENS,1,Michaels Stores Inc.,,,,,,,,,10362191 GLITTER GEL GOLD 8OZ,GLITTER GEL GOLD 8OZ,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,1.2980,1.9900,Y,1.6050,2016-06-19 00:00:00,3,7291,78775,7950197,COLART AMERICAS (IMPORT),Y,16,4,N,8,,
311477,15691,2035,86,51,12,1,-1,-1,S,2004-02-27 00:00:00,N,2017-01-16 00:00:00,10360021,206,0206 BABY INVITATIONS,68,068 BABY,25,25 WEDDING,10,RIBBON &  WEDDING,6,CELEBRATIONS,1,Michaels Stores Inc.,,,,,,,,,10360021 ANNOUNCEMENT 50CT IT'S A BOY,ANNOUNCEMENT 50C,Staple Sku,,CA,Case,2016-03-19 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,8.3700,0.0100,Y,9.8908,2008-03-03 00:00:00,3,5492,48648,70201,GARTNER STUDIOS INC,Y,3,3,N,9,9,
806845,16505,1837,103,43,2,1,-1,-1,P,2014-02-19 00:00:00,Y,2016-02-26 00:00:00,10360349,638,0638 THANKSGIVING,49,049 SEASONAL,51,51 KIDS CRAFTS,7,KIDS CRAFTS,2,KIDS/TWEENS,1,Michaels Stores Inc.,,,,,,,,,10360349 D-PIAB 2014 HARVEST,D-PIAB 2014 HARVEST,Displayer Pack,,,,4444-04-04 00:00:00,N,,N,,Y,,Seasonal,Yes,Warehouse,Active,Buyer,No,0,N,9.8000,,N,9.8000,,3,7600,82936,HARVEST PIAB,TEST RITE - KINGS BALL INTERNA,Y,4,1,,,,
310997,25637,2477,99,44,3,1,-1,-1,S,2004-02-16 00:00:00,N,2016-05-07 00:00:00,10359523,9604,9604 12X12 ALBUMS,816,816 SCRAPBOOK ALBUMS,43,43 PAPER, CARDS & ALBUM,8,PAPERCRAFTING,3,PAPERCRAFTING,1,Michaels Stores Inc.,,,,,,,,,10359523 SCRAPBOOK 12X12 BRIANNA,SCRAPBOOK 12X12,Staple Sku,,CA,Case,2016-05-07 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,12.1600,0.0100,Y,14.3694,2009-08-09 00:00:00,3,4569,36191,532397,K AND COMPANY,Y,12,3,A,,1,
806365,13382,1756,81,86,13,1,-1,-1,S,2014-02-17 00:00:00,Y,2016-11-04 00:00:00,10359656,7525,7525 BU CMAS FOLIAGE STEM,230,230 STEMS,20,20 FLORAL STEMS/BUSHES,17,FLORAL,10,SEASONAL,1,Michaels Stores Inc.,,,,,,,,,10359656 STEM SNOWFLAKE WHITE/SILVER A2,STEM SNOWFLAKE WHITE,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Seasonal,Yes,Warehouse,Discontinued,Buyer,No,0,N,0.6900,0.4900,Y,0.8709,2015-12-04 00:00:00,1,896,1770,88A0513-00,GOLD FUTURE INDUSTRIAL LTD,Y,240,24,,,,
310517,14051,1700,123,58,13,1,-1,-1,S,2004-02-02 00:00:00,Y,2016-02-26 00:00:00,10359024,8858,8858 FALL DA,133,133 BASKET COLLECTIONS,77,77 BASKETS & STORAGE,20,ACCENTS & CONTAINERS,10,SEASONAL,1,Michaels Stores Inc.,,,,,,,,,10359024 BSKT SM COPPER BEADED,BSKT SM COPPER B,Staple Sku,,EA,Each,4444-04-04 00:00:00,,,,,,,Seasonal,Yes,Warehouse,Discontinued,Buyer,No,0,N,1.2000,0.0100,Y,1.6442,2014-12-11 00:00:00,3,6374,56461,JM643316S,JM MANUFACTURING LTD,Y,1,1,,,,
79394,25223,2450,82,66,1,1,-1,-1,S,1990-01-01 00:00:00,Y,2016-05-13 00:00:00,10093130,9212,9212 LACE TRIM,788,788 LEATHER,21,21 HOBBY CRAFTS,5,UNFINISHED SURFACES,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10093130 ROUND LACE 1MM X 1.8M BLACK,ROUND LACE 1MM X 1.8,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Active,Rebuyer,No,1218,Y,0.6800,2.9900,N,0.8036,,3,7341,79652,RL0102 0112M,SILVER CREEK LEATHER COMPANY,Y,288,6,N,9,,
310037,18384,1586,116,82,4,1,-1,-1,S,2004-01-20 00:00:00,N,2017-01-19 00:00:00,10358525,2543,2543 LC SPECIALTY,10,010 CLASS NOT IN USE,70,70 PAINTING ART SUPPLIE,16,ART SUPPLIES,4,FINE ART & FRAMES,1,Michaels Stores Inc.,,,,,,,,,10358525 BRUSH WHT NYLN BRIGHT LH 12,BRUSH WHT NYLN B,Staple Sku,,CA,Case,2016-03-19 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,5.9100,0.0100,Y,6.9838,2009-12-27 00:00:00,3,1425,205,525B-12,LOEW-CORNELL, INC,Y,36,3,N,8,8,
303797,12781,1895,99,44,3,1,-1,-1,S,2003-09-12 00:00:00,N,2016-05-06 00:00:00,10351896,6378,6378 12X12 O/S SPECIALTY ,78,078 O/S PAPER,43,43 PAPER, CARDS & ALBUM,8,SCRAPBOOKING,3,PAPERCRAFTING,1,Michaels Stores Inc.,,,,,,,,,10351896 PPR-12X12 LA PINK WHIMSY,PPR-12X12 LA PIN,Staple Sku,,CA,Case,2015-07-25 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,0.2800,0.0500,Y,0.3273,2005-12-21 00:00:00,3,2809,26591,LASP003,EK SUCCESS LTD,Y,25,25,,,,
802525,26699,1607,85,51,12,1,-1,-1,S,2014-01-16 00:00:00,Y,2017-01-23 00:00:00,10348606,712,0712 $6.99 CMAS RIBBON,104,104 FASHION RIBBON,24,24 RIBBON,10,RIBBON &  WEDDING,6,CELEBRATIONS,1,Michaels Stores Inc.,,,,,,,,,10348606 RBN CMAS GLTR FERN LV 1.5IN25F,RBN CMAS GLTR FERN L,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Seasonal,Yes,Warehouse,Discontinued,Buyer,No,0,N,0.7600,0.6900,Y,0.9944,2016-04-19 00:00:00,3,7445,81347,MK802-24,LI & FUNG - NOVELTY HANDICRAFT,Y,6,6,,,,
295637,15260,1945,100,44,3,1,-1,-1,S,2003-03-21 00:00:00,Y,2016-09-01 00:00:00,10343105,9999,9999 DISCONTINUED,997,997 DISCONTINUED,44,44 STICKERS & EMBELLISH,8,PAPERCRAFTING,3,PAPERCRAFTING,1,Michaels Stores Inc.,,,,,,,,,10343105 3D DECOR MD-WIRE BABY GIRL PNK,3D DECOR MD-WIRE,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,0.8500,0.0100,Y,0.9937,2006-07-03 00:00:00,3,257,1215,TCP0936-23,HIRSCHBERG SCHUTZ AND CO,Y,4,4,,,,
282197,12840,1876,107,60,2,1,-1,-1,S,2002-08-21 00:00:00,N,2016-05-23 00:00:00,10328498,6491,6491 SUBCLASS NOT IN USE,64,064 EMBELLISHMENTS,56,56 APPAREL CRAFTS,4,APPAREL CRAFTS,2,KIDS/TWEENS,1,Michaels Stores Inc.,,,,,,,,,10328498 AP-CARTOON BUTTERFLY TURQUOISE,AP-CARTOON BUTTE,Staple Sku,,CA,Case,2015-06-13 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,0.9500,0.0100,Y,1.1106,2004-08-30 00:00:00,3,257,1215,DD11480-40,HIRSCHBERG SCHUTZ AND CO,Y,3,3,,,,
281717,15262,1947,104,26,1,1,-1,-1,S,2002-08-20 00:00:00,Y,2015-02-12 00:00:00,10327989,9999,9999 SUBCLASS NOT IN USE,997,997 DISCONTINUED,52,52 JEWELRY,1,JEWELRY & BEADS,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10327989 BDS GLS WIRE WRAP EMERALD,BDS GLS WIRE WRA,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,1.0400,0.2900,Y,1.2290,2009-04-12 00:00:00,3,104,1080,67279,HALCRAFT USA INC,Y,180,3,N,8,8,
793885,18774,1833,122,78,1,1,-1,-1,S,2013-10-25 00:00:00,Y,2016-04-22 00:00:00,10323970,2929,2929 ART TECHNIQUES,45,045 ART,76,76 BOOKS,6,BOOKS & MAGAZINES,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10323970 BK THE ART OF CARTOONING &,BK THE ART OF CARTOO,Staple Sku,,EA,Each,4444-04-04 00:00:00,,,,,,,Basic,Yes,Direct,Active,Buyer,No,0,N,11.5710,19.9500,N,13.5268,,3,8016,86588,9781600583636,BAKER AND TAYLOR INC,Y,1,1,,,,
791005,26711,1607,85,51,12,1,-1,-1,S,2013-09-20 00:00:00,Y,2017-01-16 00:00:00,10317939,724,0724 NEON RIBBON,104,104 FASHION RIBBON,24,24 RIBBON,10,RIBBON &  WEDDING,6,CELEBRATIONS,1,Michaels Stores Inc.,,,,,,,,,10317939 RBN NEON STN SHOCK PNK 7/8 7YD,RBN NEON STN SHOCK P,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Seasonal,Yes,Warehouse,Active,Buyer,No,0,N,1.1800,1.0000,N,1.3944,,3,6853,67699,MS08722-606,MOREX CORP,N,8,8,,,,
783564,20461,1701,124,94,13,1,-1,-1,S,2013-06-21 00:00:00,Y,2016-03-24 00:00:00,10303645,4545,4545 TABLETOP-VILLA ESTAT,135,135 TABLETOP,78,78 HOME ACCENTS,27,HOME ACCENTS,10,SEASONAL,1,Michaels Stores Inc.,,,,,,,,,10303645 PLANTER5.75"H CERAMIC TXTRED,PLANTER5.75"H CERAMI,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Seasonal,Yes,Warehouse,Active,Buyer,No,0,N,1.7700,0.9900,Y,2.3927,2014-06-29 00:00:00,3,7667,83659,FF10966A4,HK TONG CHENG LIMITED,Y,24,4,,,,
257476,15265,1950,107,60,2,1,-1,-1,S,2001-08-01 00:00:00,N,2016-02-08 00:00:00,10301222,9999,9999 DISCONTINUED,997,997 DISCONTINUED,56,56 APPAREL CRAFTS,4,APPAREL CRAFTS,2,KIDS/TWEENS,1,Michaels Stores Inc.,,,,,,,,,10301222 BUTTON WOOD  SNOWFLAKES,BUTTON WOOD  SNO,Staple Sku,,CA,Case,2015-05-16 00:00:00,,,,,,,Seasonal,Yes,Direct,Discontinued,Buyer,No,0,N,0.8500,0.0100,Y,0.9937,2003-11-13 00:00:00,3,1215,1981,61XB1,CRAFTY PRODUCTIONS,Y,3,3,,,,
760524,18295,1699,123,58,13,1,-1,-1,D,2012-12-12 00:00:00,Y,2016-02-26 00:00:00,10274190,2456,2456 BTS,131,131 STORAGE,77,77 BASKETS & STORAGE,20,ACCENTS & CONTAINERS,10,SEASONAL,1,Michaels Stores Inc.,,,,,,,,,10274190 DP-S/3 NYLON RECT STRG W/HNDL,DP-S/3 NYLON RECT STRG W/HNDL,Display Pack,,,,4444-04-04 00:00:00,N,,N,,Y,,Seasonal,Yes,Warehouse,Discontinued,Buyer,No,0,N,14.6000,,N,14.6000,,3,7663,83626,WL2-2098,TEST RITE - LINYI VANLONG HAND,Y,1,1,,,,
252196,11885,1584,97,3,1,1,-1,-1,S,2001-05-03 00:00:00,N,2016-03-05 00:00:00,10220878,3604,3604 CNTD CRSS STITCH KTS,10,010 STITCHERY KITS,41,41 STITCHERY,3,NEEDLE CRAFTS,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10220878 PICT XSTCH CELEBRATION OF LIF,PICT XSTCH CELEBRA,Staple Sku,,EA,Each,2016-03-05 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,7.9200,1.9900,Y,9.2587,2008-05-05 00:00:00,3,5565,495,51255,CANDAMAR DESIGN,Y,1,1,,,,
248356,12663,1909,105,53,1,1,-1,-1,S,2001-03-27 00:00:00,N,2015-04-29 00:00:00,10111995,6143,6143 CANDY MOLDS,83,083 CANDY MAKING,54,54 FOOD CRAFTING,12,FOOD CRAFT,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10111995 CANDY MOLD GIFT TRUFFLES,CANDY MOLD GIFT,Staple Sku,,CA,Case,2015-04-18 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,N,1.0600,0.0100,Y,1.2392,2003-04-05 00:00:00,3,1338,20158,2115-1728,WILTON INDUSTRIES (DOMESTIC),Y,3,3,,,,
778764,26511,2263,117,81,12,1,-1,-1,S,2013-05-13 00:00:00,Y,2017-01-17 00:00:00,10297062,537,0537 ST PATS,723,723 PARTY GOODS ACCESSOR,71,71 PACKAGING & PARTY,23,PACKAGING & PARTY,6,CELEBRATIONS,1,Michaels Stores Inc.,,,,,,,,,10297062 TABLE COVER,TABLE COVER,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Seasonal,Yes,Warehouse,Discontinued,Buyer,No,0,N,0.7000,0.0100,Y,0.8673,2014-06-18 00:00:00,3,8279,89840,RS5270-1,NINGBO GUANGBO IMPORT AND EXPO,Y,72,6,,,,
770604,17802,1792,107,60,2,1,-1,-1,P,2013-03-12 00:00:00,Y,2016-05-13 00:00:00,10286748,1995,1995 BREAST CANCER,30,030 WEARABLE ACCESSORIES,56,56 APPAREL CRAFTS,4,APPAREL CRAFTS,2,KIDS/TWEENS,1,Michaels Stores Inc.,,,,,,,,,10286748 D-PDQ BCA XL WHT HEL SFTY PNK,D-PDQ BCA XL WHT HEL SFTY PNK,Displayer Pack,,,,4444-04-04 00:00:00,N,,N,,Y,,Seasonal,Yes,Warehouse,Active,Buyer,No,0,N,43.1100,,N,43.1100,,3,8451,93423,BCA4XLPWH,GILDAN USA INC,Y,1,1,,,,
771564,19053,2186,83,86,13,1,-1,-1,S,2013-03-18 00:00:00,Y,2015-08-21 00:00:00,10287945,3200,3200 CMAS DRIED COMPONENT,961,961 DRIEDS,22,22 GREENERY,17,FLORAL,10,SEASONAL,1,Michaels Stores Inc.,,,,,,,,,10287945 LATA BALL A2,LATA BALL A2,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Seasonal,Yes,Warehouse,Discontinued,Buyer,No,0,N,0.8500,0.0100,Y,1.0851,2014-06-17 00:00:00,3,7969,86196,SF21650M13,TEST RITE - SIBAFLOR NATURAL D,Y,18,18,,,,
216676,13670,1639,85,51,8,1,-1,-1,S,1990-01-01 00:00:00,N,2017-01-16 00:00:00,10276775,8067,8067 ROSES,112,112 RIBBON ACCESSORIES,24,24 RIBBON,10,RIBBON &  WEDDING,7,FLORAL/WEDDING,1,Michaels Stores Inc.,,,,,,,,,10276775 SM SHEER ROSE 815 CREAM,SM SHEER ROSE,Staple Sku,,CA,Case,2015-05-09 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,0.3300,0.0100,Y,0.3858,2004-06-21 00:00:00,3,4295,345,10-3249-815,BERWICK INDUSTRIES  OFFRAY INC,Y,6,6,,,,
753324,26553,2500,131,43,2,1,-1,-1,S,2012-09-25 00:00:00,Y,2016-11-10 00:00:00,10265715,574,0574 FUN FINDS 2013 PP,849,849 IMPULSE TOYS,58,58 KIDS ACTIVITY/HOBBY,7,KIDS CRAFTS,2,KIDS/TWEENS,1,Michaels Stores Inc.,,,,,,,,,10265715 SILICONE NECKLACE ASTMNT,SILICONE NECKLACE AS,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Seasonal,Yes,Warehouse,Discontinued,Buyer,No,0,N,1.3000,0.0100,Y,1.5580,2013-09-30 00:00:00,3,8271,89724,VS S05,LI & FUNG - V AND S INTERNATIO,Y,144,6,,,,
755724,20306,1715,121,73,12,1,-1,-1,P,2012-10-22 00:00:00,Y,2017-01-16 00:00:00,10268310,4401,4401 SUBCLASS NOT IN USE,15,015 FASHION,75,75 IMPULSE,11,IMPULSE,6,CELEBRATIONS,1,Michaels Stores Inc.,,,,,,,,,10268310 D-HOLIDAY BLINGKERS AST,D-HOLIDAY BLINGKERS AST,Displayer Pack,,,,4444-04-04 00:00:00,N,,N,,Y,,Seasonal,Yes,Warehouse,Active,Buyer,No,0,N,54.0000,,N,54.0000,,1,6642,63774,37100PDQ,LEAP YEAR PUBLISHING LLC,Y,1,1,,,,
749004,20670,2221,100,44,3,1,-1,-1,S,2012-08-10 00:00:00,Y,2016-09-01 00:00:00,10260312,4738,4738 CRAFT TAPE-SBA,443,443 TAPES & TRIMS,44,44 STICKERS & EMBELLISH,8,PAPERCRAFTING,3,PAPERCRAFTING,1,Michaels Stores Inc.,,,,,,,,,10260312 TURQUOISE SOLID RIBBONS,TURQUOISE SOLID RIBB,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,0.8300,0.9900,Y,1.0600,2013-10-26 00:00:00,3,7451,81470,MK6-019,JOYFUL LANDS CRAFT CO LTD,Y,288,3,N,2,,
204676,15721,1945,100,44,3,1,-1,-1,S,1990-01-01 00:00:00,Y,2016-09-01 00:00:00,10262403,9998,9998 DISCONTINUED-NON PI,997,997 DISCONTINUED,44,44 STICKERS & EMBELLISH,8,PAPERCRAFTING,3,PAPERCRAFTING,1,Michaels Stores Inc.,,,,,,,,,10262403 DIE CUT ALPH 1-1/4" BLCK UP D,DIE CUT ALPH,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,N,0.1000,0.0100,Y,0.1169,2005-10-03 00:00:00,3,4232,34155,13646,MAKING MEMORIES INC,N,24,24,,,,
170835,12053,1900,105,53,1,1,-1,-1,S,1990-01-01 00:00:00,N,2016-04-20 00:00:00,10221717,4221,4221 BASIC PANS,80,080 PANS & MOLDS,54,54 FOOD CRAFTING,12,FOOD CRAFT,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10221717 HALF ROUND 18"_,HALF ROUND 18",Staple Sku,,CA,Case,2015-05-02 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,N,9.0000,0.0100,Y,10.5212,1999-05-20 00:00:00,3,1338,20158,CLEARANCE,WILTON INDUSTRIES (DOMESTIC),Y,3,3,,,,
707003,9434,1464,110,90,13,1,-1,-1,S,2011-09-23 00:00:00,Y,2015-09-11 00:00:00,10218037,7607,7607 SUPPLIES,101,101 CUSTOM FLORAL SUPPLY,61,61 CUSTOM FLORAL,25,CUSTOM FLORAL,10,SEASONAL,1,Michaels Stores Inc.,,,,,,,,,10218037 DECO MESH - LIME GREEN,DECO MESH - LIME GRE,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Supplies,No,Warehouse,Active,Rebuyer,Yes,0,N,2.0000,0.0000,N,2.2509,2012-08-17 00:00:00,3,7908,85655,MSH1722,CANVAS CORP LLC,N,20,1,N,7,,
714683,24299,2421,105,77,12,1,-1,-1,S,2011-11-30 00:00:00,Y,2016-04-20 00:00:00,10225934,8294,8294 8 OZ FONDANT,696,696 MIXES, MELTS & FROST,54,54 FOOD CRAFTING,12,FOOD CRAFT,6,CELEBRATIONS,1,Michaels Stores Inc.,,,,,,,,,10225934 CN FD FONDANT BLUE 8OZ,CN FD FONDANT BLUE 8,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,3.8500,6.9900,Y,4.5007,2015-05-10 00:00:00,2,7926,85781,93673,GARTNER STUDIOS INC (DUFF LICE,Y,3,3,,,,
142515,25212,1695,92,56,13,1,-1,-1,S,1990-01-01 00:00:00,Y,2016-04-20 00:00:00,10189471,9201,9201 MARDI GRAS NOVELTIES,13,013 NOVELTIES,30,30 SPRING HOLIDAY,19,MINI SEASON,10,SEASONAL,1,Michaels Stores Inc.,,,,,,,,,10189471 MG SATIN CAT MASK,MG SATIN CAT MAS,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Seasonal,Yes,Direct,Discontinued,Buyer,No,0,N,5.5000,0.0100,Y,5.9950,2009-02-25 00:00:00,1,1810,21203,38004,LOUISIANA GIFTS AND GALLERY IN,Y,12,12,,,,
142995,12165,1580,81,34,8,1,-1,-1,S,1990-01-01 00:00:00,N,2015-08-21 00:00:00,10190198,4534,4534 SLK GARDN SHP FLORAL,10,010 SBA STEMS,20,20 FLORAL STEMS/BUSHES,17,FLORAL,7,FLORAL/WEDDING,1,Michaels Stores Inc.,,,,,,,,,10190198 SLK MED SUNFLOWER CR/WH,SLK MED SUNFLWR,Staple Sku,,CA,Case,2014-04-05 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,N,0.8051,0.0100,Y,0.9412,2003-01-25 00:00:00,3,1879,21295,GDS024-CR/WH,ALLSTATE FLORAL & CRAFT INC.,Y,24,24,,,,
71714,25499,1797,108,66,1,1,-1,-1,S,1990-01-01 00:00:00,Y,2016-04-15 00:00:00,10083202,9491,9491 LARGE WOOD,308,308 LARGE WOOD,57,57 UNFINISHED SURFACES,5,UNFINISHED SURFACES,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10083202 WD-STOOL 30" UNF HARD TOP,WD-STOOL 30" UNF,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,13.5000,2.9900,Y,15.9529,2011-05-06 00:00:00,3,5647,50775,1530 UNF,THE EMERSON GROUP INC,Y,2,2,A,,7,
49873,12323,1714,108,48,2,1,-1,-1,S,1990-01-01 00:00:00,N,2016-02-26 00:00:00,10057602,5158,5158 SUBCLASS NOT IN USE,15,015 CLASS NOT IN USE,57,57 UNFINISHED SURFACES,5,WOOD CRAFTS,2,KIDS/TWEENS,1,Michaels Stores Inc.,,,,,,,,,10057602 CLASSIC DIMNSION SERIES 2,CLASSIC DIMNSION,Staple Sku,,CA,Case,2010-09-25 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,N,1.2600,0.0100,Y,1.4889,2003-03-15 00:00:00,3,74,1054,16201,WALNUT HOLLOW FARMS,Y,24,6,A,,,
41233,12483,1578,103,43,2,1,-1,-1,S,1990-01-01 00:00:00,N,2016-02-26 00:00:00,10048049,5788,5788 SUBCLASS NOT IN USE,1,001 GNRAL CRFTS QUIK CDE,51,51 KIDS CRAFTS,7,KIDS CRAFTS,2,KIDS/TWEENS,1,Michaels Stores Inc.,,,,,,,,,10048049 WOOD BEADS 12MM RND NTRAL,WOOD BEADS 12MM R,Staple Sku,,CA,Case,2011-11-03 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,N,0.3400,0.0900,Y,0.3975,2002-02-28 00:00:00,3,824,175,2721-PE-006,WESTRIM CRAFTS,Y,6,6,,,,
28753,15247,1932,85,27,4,1,-1,-1,S,1990-01-01 00:00:00,N,2017-01-16 00:00:00,10033959,9999,9999 DISCONTINUED,997,997 DISCONTINUED,24,24 RIBBON,10,RIBBON &  WEDDING,4,FRAMES & DECOR,1,Michaels Stores Inc.,,,,,,,,,10033959 RBN GLIT LEAVES W/GRN 1.5 12F,RBN GLIT LEAVE,Staple Sku,,EA,Each,2007-10-28 00:00:00,,,,,,,Seasonal,Yes,Direct,Discontinued,Buyer,No,0,N,1.1200,1.0000,Y,1.3093,2002-06-21 00:00:00,1,4295,345,WI6090MC GLT1.5,BERWICK INDUSTRIES / OFFRAY,,Y,1,1,,,,
3313,11465,1607,85,51,12,1,-1,-1,S,1990-01-01 00:00:00,Y,2017-01-16 00:00:00,10005177,1683,1683 MONOGRAM RIBBON,104,104 FASHION RIBBON,24,24 RIBBON,10,RIBBON &  WEDDING,6,CELEBRATIONS,1,Michaels Stores Inc.,,,,,,,,,10005177 RBBN FACEING 3 X0622 BLUE,RBBN FACEING 3,Staple Sku,,EA,Each,4444-04-04 00:00:00,,,,,,,Seasonal,Yes,Direct,Discontinued,Buyer,No,0,N,1.2800,1.0000,Y,1.4964,2002-05-15 00:00:00,3,578,1552,40247,BERWICK OFFRAY LLC (LICENSING),Y,1,1,,,,
309557,13214,1601,96,3,1,1,-1,-1,S,2004-01-07 00:00:00,Y,2016-02-26 00:00:00,10358026,7351,7351 YARN ACCESSORIES,102,102 YARN ACCESSORIES,40,40 YARN,3,NEEDLE CRAFTS,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10358026 MARKERS SPLIT RING,MARKERS SPLIT RI,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,1.0600,0.2700,Y,1.2526,2011-05-10 00:00:00,3,928,180,3002,LION BRAND YARN,Y,100,5,N,8,8,
309077,19656,2216,104,26,1,1,-1,-1,S,2003-12-30 00:00:00,N,2016-05-19 00:00:00,10357528,3784,3784 SUBCLASS NOT IN USE,381,381 CLASS NOT IN USE,52,52 JEWELRY,1,JEWELRY & BEADS,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10357528 BEAD-GLASS PILL PERIDOT MIX,BEAD-GLASS PILL,Staple Sku,,CA,Case,2015-08-01 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,0.7900,0.0100,Y,0.9235,2006-12-04 00:00:00,3,1854,21252,2949522,COUSIN CORPORATION OF AMERICA,Y,6,6,,,,
308597,18773,1833,122,78,1,1,-1,-1,S,2003-12-19 00:00:00,Y,2016-04-14 00:00:00,10357036,2928,2928 WATERCOLOR/PAINTING,45,045 ART,76,76 BOOKS,6,BOOKS & MAGAZINES,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10357036 BK PLEIN AIR PAINTING IN WA,BK PLEIN AIR PAI,Staple Sku,,EA,Each,4444-04-04 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,N,5.1900,4.3900,Y,6.0672,2010-03-13 00:00:00,3,7134,75523,LA05041,HDA INC,Y,1,1,,,,
337638,15251,1936,90,55,13,1,-1,-1,S,2005-11-28 00:00:00,Y,2016-02-26 00:00:00,10394746,9999,9999 DISCONTINUED,997,997 DISCONTINUED,29,29 CMAS DECOR,18,CHRISTMAS,10,SEASONAL,1,Michaels Stores Inc.,,,,,,,,,10394746 STORE USE DUMMY D9C36S2794,STORE USE DUMMY,Staple Sku,,EA,Each,4444-04-04 00:00:00,,,,,,,U,Yes,Direct,Discontinued,Buyer,No,0,N,0.0000,0.0000,Y,0.0000,2005-11-28 00:00:00,1,1,1,STORE USE DUMMY,MICHAELS DISTRIBUTION CENTER #,Y,1,1,,,,
805405,15373,1727,121,73,12,1,-1,-1,S,2014-02-10 00:00:00,Y,2017-01-16 00:00:00,10356350,9290,9290 CANADA SPRING/SUMMER,17,017 CANDY,75,75 IMPULSE,11,IMPULSE,6,CELEBRATIONS,1,Michaels Stores Inc.,,,,,,,,,10356350 CN-SS CNDY STICKER STAMPER,CN-SS CNDY STICKER S,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Seasonal,Yes,Direct,Active,Buyer,No,0,N,3.1200,4.9900,Y,3.2776,2014-08-31 00:00:00,2,8489,94587,021923,CORE MARK INTERNATIONAL INC,Y,12,12,,,,
825565,19550,2209,104,26,1,1,-1,-1,P,2014-09-16 00:00:00,Y,2015-02-27 00:00:00,10395730,3685,3685 STRUNG BEADS 5.99,261,261 STRUNG BEADS,52,52 JEWELRY,1,JEWELRY & BEADS,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10395730 D-INLINE PIAB,D-INLINE PIAB,Displayer Pack,,,,4444-04-04 00:00:00,N,,N,,Y,,Seasonal,Yes,Warehouse,Active,Buyer,No,0,N,16.4100,,N,16.4100,,3,104,1080,83166,HALCRAFT USA INC,Y,1,1,,,,
337158,12461,1768,83,86,13,1,-1,-1,S,2005-11-28 00:00:00,Y,2016-07-23 00:00:00,10394259,5725,5725 GRAPEVINE DECOR,26,026 STRAW/VINE,22,22 GREENERY,17,FLORAL,10,SEASONAL,1,Michaels Stores Inc.,,,,,,,,,10394259 GRAPEVINE BALL 10",GRAPEVINE BALL 1,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Active,Rebuyer,No,853,Y,1.4400,13.9900,N,2.8887,2013-05-10 00:00:00,3,5447,47859,23011,PANACEA PRODUCTS (IMPORT),Y,4,4,N,2,2,
306677,19954,2232,97,3,1,1,-1,-1,S,2003-11-05 00:00:00,Y,2016-03-26 00:00:00,10354995,4068,4068 SCISSORS,601,601 SEWING NOTIONS,41,41 STITCHERY,3,NEEDLE CRAFTS,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10354995 SCISSOR 6 1/2" FABRIC RUB GRIP,SCISSOR 6 1/2" F,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,3.5000,0.0100,Y,4.0916,2006-07-03 00:00:00,1,1806,212,02073,DYNO MERCH,Y,3,3,,,,
306197,18792,2116,108,66,1,1,-1,-1,S,2003-10-28 00:00:00,N,2016-07-28 00:00:00,10354498,2947,2947 SUBCLASS NOT IN USE,282,282 CLASS NOT IN USE,57,57 UNFINISHED SURFACES,5,UNFINISHED SURFACES,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10354498 UNF RESIN PHOTO HLDR SCHOOL,UNF RESIN PHOTO,Staple Sku,,CA,Case,2016-03-19 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,1.1000,0.0100,Y,1.3614,2006-07-31 00:00:00,3,697,1653,21682,DON MECHANIC ENTERPRISE IMPORT,Y,36,6,A,,,
336198,16585,1810,104,26,1,1,-1,-1,S,2005-11-03 00:00:00,N,2016-05-25 00:00:00,10392915,720,0720 SUBCLASS NOT IN USE,35,035 CLASS NOT IN USE,52,52 JEWELRY,1,JEWELRY & BEADS,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10392915 FLTBK HTFX 4MM MIX NUMBER 2,FLTBK HTFX 4MM M,Staple Sku,,CA,Case,2015-08-22 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,1.6200,0.0100,Y,1.8938,2007-04-23 00:00:00,3,5295,45275,112284190,PURE ALLURE INC,Y,6,6,,,,
825085,23273,1829,48,54,4,1,-1,-1,S,2014-09-11 00:00:00,Y,2017-01-29 00:00:00,10395219,7273,7273 ARTIST REVIVAL,45,045 VIEWPOINT DECOR,34,34 WALL DECOR,14,FRAMES & WALL DECOR,4,FINE ART & FRAMES,1,Michaels Stores Inc.,,,,,,,,,10395219 WOOD LETTER "P" ARTIST REVIVAL,WOOD LETTER "P" ARTI,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,1233,Y,4.3800,14.9900,Y,5.6101,2017-01-29 00:00:00,3,7676,83785,14QH180C5D2,LI & FUNG - EASTOWN INTL INDUS,Y,16,2,N,2,,
824125,19549,2209,104,26,1,1,-1,-1,S,2014-09-02 00:00:00,Y,2016-07-25 00:00:00,10393278,3684,3684 STRUNG BEADS 3.99,261,261 STRUNG BEADS,52,52 JEWELRY,1,JEWELRY & BEADS,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10393278 BDS GLS BEACH MIX 6MM ROUND,BDS GLS BEACH MIX 6M,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,0.8075,0.3900,Y,0.9293,2015-06-28 00:00:00,3,7328,79322,82843,HALCRAFT USA (IMPORT),Y,180,3,N,9,,
335718,24441,2139,100,44,3,1,-1,-1,S,2005-10-21 00:00:00,Y,2016-09-01 00:00:00,10392153,8436,8436 WEDDING-SBA,527,527 THEMED STICKERS,44,44 STICKERS & EMBELLISH,8,PAPERCRAFTING,3,PAPERCRAFTING,1,Michaels Stores Inc.,,,,,,,,,10392153 STKR CHURCH TOUCH OF JOLEES,STKR CHURCH TOUC,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,0.8070,0.9900,Y,0.9536,2011-03-30 00:00:00,3,2809,26591,SPJJ050,EK SUCCESS LTD,Y,432,3,N,9,9,
305717,13826,1649,50,54,4,1,-1,-1,S,2003-10-14 00:00:00,Y,2016-04-26 00:00:00,10353900,8455,8455 TD EXP BROWN,114,114 FASHION TTOP,36,36 TABLE TOP FRAMES,14,FRAMES & WALL DECOR,4,FINE ART & FRAMES,1,Michaels Stores Inc.,,,,,,,,,10353900 FRM-3.5X5 WALNUT W/BLK I/O RIM,FRM-3.5X5 WALNUT,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,2.7600,0.0100,Y,3.2615,2013-06-23 00:00:00,3,6094,790,7433-35180,THE WESTON GALLERY,Y,36,3,N,9,9,
804925,9434,1464,110,90,13,1,-1,-1,S,2014-02-04 00:00:00,Y,2016-04-19 00:00:00,10354948,7607,7607 SUPPLIES,101,101 CUSTOM FLORAL SUPPLY,61,61 CUSTOM FLORAL,25,CUSTOM FLORAL,10,SEASONAL,1,Michaels Stores Inc.,,,,,,,,,10354948 RBN CMAS GD SWL SC 2.5"50Y,RBN CMAS GD SWL SC 2,Staple Sku,,EA,Each,4444-04-04 00:00:00,,,,,,,Seasonal,Yes,Warehouse,Discontinued,Buyer,No,0,N,5.5200,9.9900,N,7.2386,,3,7445,81347,A13512-01,LI & FUNG - NOVELTY HANDICRAFT,N,1,1,,,,
253156,12840,1876,107,60,2,1,-1,-1,S,2001-05-10 00:00:00,N,2016-05-23 00:00:00,10269365,6491,6491 SUBCLASS NOT IN USE,64,064 EMBELLISHMENTS,56,56 APPAREL CRAFTS,4,APPAREL CRAFTS,2,KIDS/TWEENS,1,Michaels Stores Inc.,,,,,,,,,10269365 APP BEADED SQR W/EMBRDRY TURQ,APP BEADED SQR W,Staple Sku,,CA,Case,2016-05-21 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,0.9500,1.0000,Y,1.1106,2004-05-03 00:00:00,3,257,1215,DB 7803-40,HIRSCHBERG SCHUTZ AND CO,Y,6,6,,,,
304757,17655,2115,108,66,1,1,-1,-1,S,2003-09-25 00:00:00,N,2016-07-13 00:00:00,10352921,1854,1854 SUBCLASS NOT IN USE,281,281 LETTERS,57,57 UNFINISHED SURFACES,5,UNFINISHED SURFACES,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10352921 MINI CLOTHS PIN PRIMARY COLORS,MINI CLOTHS PIN,Staple Sku,,CA,Case,2016-03-19 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,1.1200,0.0100,Y,1.3629,2011-09-04 00:00:00,3,7276,78327,349202,NEW IMAGE COMPANY (IMPORT),Y,72,3,N,7,,
304277,4664,369,129,23,6,1,-1,-1,S,2003-09-19 00:00:00,Y,2012-10-22 00:00:00,10352388,9090,9090 ALL DEPARTMENTS,55,055 MERCHANDISING AIDES,90,90 SUPPLIES,98,NON-MERCHANDISING,9,NON-MERCHANDISING,1,Michaels Stores Inc.,,,,,,,,,10352388 BL HLDR 6" / 12" PEGS 100PK,BL HLDR 6" / 12",Staple Sku,,EA,Each,4444-04-04 00:00:00,,,,,,,Supplies,No,Direct,Active,Buyer,Yes,1232,N,8.4000,0.0000,N,9.8198,,3,6874,68367,175399,BUNZL EXTRUSION, INC,N,1,1,,,,
303317,15260,1945,100,44,3,1,-1,-1,S,2003-09-09 00:00:00,N,2016-09-01 00:00:00,10351398,9999,9999 DISCONTINUED,997,997 DISCONTINUED,44,44 STICKERS & EMBELLISH,8,SCRAPBOOKING,3,PAPERCRAFTING,1,Michaels Stores Inc.,,,,,,,,,10351398 STICKER CRYSTAL HEARTS 7PC,STICKER CRYSTAL,Staple Sku,,CA,Case,2015-07-25 00:00:00,,,,,,,Seasonal,Yes,Direct,Discontinued,Buyer,No,0,N,1.2000,2.9900,N,1.4028,,3,6610,62398,1735,MARK RICHARDS ENTERPRISES,Y,6,6,,,,
804445,26487,2261,117,81,12,1,-1,-1,S,2014-01-31 00:00:00,Y,2017-01-17 00:00:00,10353665,513,0513 HALLOWEEN PACKAGING,721,721 GIFT PACKAGING,71,71 PACKAGING & PARTY,23,PACKAGING & PARTY,6,CELEBRATIONS,1,Michaels Stores Inc.,,,,,,,,,10353665 BLACK BLK BATS ORGANZA BAG,BLACK BLK BATS ORGAN,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Seasonal,Yes,Warehouse,Discontinued,Buyer,No,0,N,0.3400,0.0900,Y,0.4400,2014-10-26 00:00:00,3,7442,81316,LP10-4,LACE FASHIONS IND CO LTD,Y,6,6,,,,
300917,12390,1876,107,60,2,1,-1,-1,S,2003-06-27 00:00:00,N,2016-05-23 00:00:00,10348820,547,0547 SUBCLASS NOT IN USE,64,064 EMBELLISHMENTS,56,56 APPAREL CRAFTS,4,APPAREL CRAFTS,2,KIDS/TWEENS,1,Michaels Stores Inc.,,,,,,,,,10348820 BTNS VP CHRISTMAS,BTNS VP CHRISTMA,Staple Sku,,CA,Case,2016-05-21 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,1.0200,1.0000,Y,1.1924,2005-11-05 00:00:00,3,1215,1981,47VB3,CRAFTY PRODUCTIONS,Y,3,3,,,,
803965,25748,2488,125,90,13,1,-1,-1,S,2014-01-28 00:00:00,Y,2016-04-14 00:00:00,10351035,9715,9715 FALL WALL DECOR,827,827 WALL DECOR,79,79 CUSTOM FLORAL PERPET,25,CUSTOM FLORAL,10,SEASONAL,1,Michaels Stores Inc.,,,,,,,,,10351035 CN FLORAL TEARDROP A2,CN FLORAL TEARDROP A,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Seasonal,Yes,Warehouse,Active,Buyer,No,0,N,9.4600,59.9900,Y,13.5397,2014-10-14 00:00:00,2,50,10347,CN13BMSI-B-171/172,UCP INTERNATIONAL CO LTD (IMPO,Y,4,4,,,,
300437,20063,2255,119,73,12,1,-1,-1,S,2003-06-13 00:00:00,Y,2017-01-16 00:00:00,10348305,4164,4164 HC RIBBON BY THE YAR,682,682 HOMECOMING,73,73 REGIONAL MERCHANDISI,11,IMPULSE,6,CELEBRATIONS,1,Michaels Stores Inc.,,,,,,,,,10348305 HC RBY UNPRNTD 03 SCARLET,HC RBY UNPRNTD 0,Staple Sku,,EA,Each,4444-04-04 00:00:00,,,,,,,Basic,Yes,Direct,Active,Buyer,Yes,87,N,5.3600,19.0000,N,6.2660,,1,1756,21118,RSEAU03SCR,CREATIVE RIBBON ETC,N,1,1,,,,
299957,13979,1828,48,54,4,1,-1,-1,S,2003-06-03 00:00:00,N,2016-08-05 00:00:00,10347796,8748,8748 TD FLAT PRINTS,43,043 SBA PRINTS/POSTERS,34,34 WALL DECOR,14,FRAMES & WALL DECOR,4,FRAMES & DECOR,1,Michaels Stores Inc.,,,,,,,,,10347796 PRNT 11X14 PINK PASSION,PRNT 11X14 PINK,Staple Sku,,EA,Each,2015-07-04 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,3.1000,0.0100,Y,3.6240,2007-07-09 00:00:00,3,2501,2384,PF1113,ARTISTREE, INC.,Y,1,1,,,,
299477,23683,2395,92,56,13,1,-1,-1,S,2003-05-21 00:00:00,N,2016-02-26 00:00:00,10347283,7692,7692 BU NOVELTIES DECOR,586,586 BU HANUKKAH,30,30 SPRING HOLIDAY,19,MINI SEASON,10,SEASONAL,1,Michaels Stores Inc.,,,,,,,,,10347283 STICKERS-ASSTD HANUKKAH(3PK),STICKERS-ASSTD H,Staple Sku,,CA,Case,2015-07-04 00:00:00,,,,,,,Seasonal,Yes,Direct,Discontinued,Buyer,No,0,N,1.0000,0.0100,Y,1.1690,2010-03-19 00:00:00,3,6539,60168,TY-14337,RITE LITE LTD,Y,12,12,,,,
298037,20064,2255,119,73,12,1,-1,-1,S,2003-05-01 00:00:00,Y,2017-01-16 00:00:00,10345664,4165,4165 HC TRINKETS,682,682 HOMECOMING,73,73 REGIONAL MERCHANDISI,11,IMPULSE,6,CELEBRATIONS,1,Michaels Stores Inc.,,,,,,,,,10345664 HC-GARLAND STAR FOIL 25' WHITE,HC-GARLAND STAR,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,87,N,1.1000,1.0000,Y,1.2859,2004-10-27 00:00:00,1,2609,24653,237540,CURRIE HOLDINGS LP DBA ACI DIS,Y,12,12,,,,
297557,13979,1828,48,54,4,1,-1,-1,S,2003-04-29 00:00:00,Y,2016-08-05 00:00:00,10345172,8748,8748 TD FLAT PRINTS,43,043 SBA PRINTS/POSTERS,34,34 WALL DECOR,14,FRAMES & WALL DECOR,4,FINE ART & FRAMES,1,Michaels Stores Inc.,,,,,,,,,10345172 PRNT 11X14 RIVR SEINE  CTY PA,PRNT 11X14 RIVR SEIN,Staple Sku,,EA,Each,4444-04-04 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,3.1000,0.0100,Y,3.6240,2011-05-28 00:00:00,3,2501,2384,4157,ARTISTREE, INC.,Y,1,1,,,,
803485,22738,1854,116,82,4,1,-1,-1,S,2014-01-23 00:00:00,Y,2016-11-19 00:00:00,10350092,6732,6732 PEBEO MIXED MEDIA,54,054 ACRYLIC PAINT,70,70 PAINTING ART SUPPLIE,16,ART SUPPLIES,4,FINE ART & FRAMES,1,Michaels Stores Inc.,,,,,,,,,10350092 VITRAIL 45ML CHARTREUSE,VITRAIL 45ML CHARTRE,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,863,Y,2.6600,7.9900,N,3.1433,,1,3357,29869,050-018US,PEBEO INC.,Y,270,3,N,8,,
296597,17982,2143,100,44,3,1,-1,-1,S,2003-04-04 00:00:00,N,2016-09-01 00:00:00,10344128,2172,2172 ALPHAS-SBA,531,531 ALPHAS (STICKERS),44,44 STICKERS & EMBELLISH,8,SCRAPBOOKING,3,PAPERCRAFTING,1,Michaels Stores Inc.,,,,,,,,,10344128 STKR CHLOE DIECUT ALPHA,STKR CHLOE DIECU,Staple Sku,,CA,Case,2015-07-04 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,1.5940,0.0100,Y,1.8634,2006-10-02 00:00:00,3,4569,36191,553002,K AND COMPANY,Y,6,6,,,,
294677,17972,2137,100,44,3,1,-1,-1,S,2003-03-13 00:00:00,N,2016-09-01 00:00:00,10342090,2162,2162 FLOWERS-SBA,525,525 FLOWERS (STICKERS),44,44 STICKERS & EMBELLISH,8,SCRAPBOOKING,3,PAPERCRAFTING,1,Michaels Stores Inc.,,,,,,,,,10342090 JOLEES BY YOU BLUE PASSION FLW,JOLEES BY YOU BL,Staple Sku,,CA,Case,2015-06-27 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,0.4284,0.0100,Y,0.5008,2007-02-26 00:00:00,3,2809,26591,JJAA135A,EK SUCCESS LTD,Y,6,6,,,,
294197,20334,2262,117,81,12,1,-1,-1,S,2003-03-11 00:00:00,Y,2017-01-17 00:00:00,10341592,4429,4429 CURL RIBBON,722,722 PACKAGING ACCESSORIE,71,71 PACKAGING & PARTY,23,PACKAGING & PARTY,6,CELEBRATIONS,1,Michaels Stores Inc.,,,,,,,,,10341592 RBN CURL 100YD HOLO RED,RBN CURL 100YD H,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,0.6000,0.3900,Y,0.7591,2014-11-09 00:00:00,3,7464,81576,SP-100L,LI & FUNG - LANEE GREAT PLASTI,Y,72,6,N,1,,
293237,13978,1828,48,54,4,1,-1,-1,S,2003-03-06 00:00:00,N,2016-08-05 00:00:00,10340577,8747,8747 TD MATTED PRINTS,43,043 SBA PRINTS/POSTERS,34,34 WALL DECOR,14,FRAMES & WALL DECOR,4,FRAMES & DECOR,1,Michaels Stores Inc.,,,,,,,,,10340577 MAT PRNT 11X14 FLAVIA BELIEVE,MAT PRNT 11X14 F,Staple Sku,,EA,Each,2015-06-27 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,4.3800,0.0100,Y,5.1203,2007-07-09 00:00:00,3,1319,20122,MAL 214,INNOVATIVE ART,Y,1,1,,,,
292757,12784,1897,99,44,3,1,-1,-1,S,2003-03-01 00:00:00,N,2016-05-06 00:00:00,10340065,6389,6389 12X12 PREMIUM PAPER ,79,079 PAPER PADS,43,43 PAPER, CARDS & ALBUM,8,SCRAPBOOKING,3,PAPERCRAFTING,1,Michaels Stores Inc.,,,,,,,,,10340065 PPR 12X12 BK TRACY PORTER,PPR 12X12 BK TRA,Staple Sku,,CA,Case,2015-06-27 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,3.0400,0.8900,Y,3.5924,2009-08-18 00:00:00,3,306,126,19339MI,COLORBOK INC.,Y,48,3,N,4,4,
801085,22763,2218,104,26,1,1,-1,-1,S,2013-12-23 00:00:00,Y,2016-06-08 00:00:00,10344685,6757,6757 OTHER COLLECTIONS,421,421 JEWELRY COLLECTIONS,52,52 JEWELRY,1,JEWELRY & BEADS,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10344685 CHAIN SML ROUND STONES GOLD,CHAIN SML ROUND STON,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Seasonal,Yes,Warehouse,Active,Buyer,No,0,N,0.8000,1.9900,N,0.9454,,3,257,1215,P28251-02,HIRSCHBERG SCHUTZ AND CO,Y,210,3,,,,
292277,16504,1769,108,66,1,1,-1,-1,S,2003-02-24 00:00:00,Y,2017-01-30 00:00:00,10339562,637,0637 SUBCLASS NOT IN USE,26,026 ALTERNATIVE SURFACES,57,57 UNFINISHED SURFACES,5,UNFINISHED SURFACES,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10339562 ORN DISPLAY BOX 5 FLAT,ORN DISPLAY BOX,Staple Sku,,EA,Each,4444-04-04 00:00:00,,,,,,,Supplies,No,Direct,Active,Buyer,No,0,N,0.0000,0.0000,N,0.0000,,3,144,1114,457994,TODAY'S DESIGN HOUSE,N,1,1,,,,
799645,20067,2256,119,73,12,1,-1,-1,S,2013-12-20 00:00:00,Y,2017-01-16 00:00:00,10339538,4168,4168 COLLEGE FOOTBALL,683,683 LICENSED SPORTS,73,73 REGIONAL MERCHANDISI,11,IMPULSE,6,CELEBRATIONS,1,Michaels Stores Inc.,,,,,,,,,10339538 PINT GLASS ARIZONA STATE,PINT GLASS ARIZONA S,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,5.4000,5.0000,Y,6.3127,2015-05-31 00:00:00,1,8367,91545,COL-AZS-1465,THE MEMORY COMPANY,Y,6,6,,,,
291797,16734,2050,132,82,4,1,-1,-1,S,2003-02-11 00:00:00,Y,2016-12-14 00:00:00,10339021,893,0893 COLOR PENCILS,10,010 PENCILS & PASTELS,81,81 DRAWING ART SUPPLIES,16,ART SUPPLIES,4,FINE ART & FRAMES,1,Michaels Stores Inc.,,,,,,,,,10339021 AL PENCILS COLORED 12 PC,AL PENCILS COLORED 1,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Active,Rebuyer,No,1240,Y,1.3900,5.9900,N,1.6719,,3,7515,82117,CP12-CEDAR3 0/ACE,LI & FUNG - ARUNA CO LTD (IMPO,Y,120,6,N,4,,
290837,13070,1599,120,59,13,1,-1,-1,S,2003-01-28 00:00:00,Y,2016-03-25 00:00:00,10337989,7033,7033 FORMALS/CAMELOTS,101,101 TAPERS,74,74 CANDLES,21,CANDLES,10,SEASONAL,1,Michaels Stores Inc.,,,,,,,,,10337989 CNDL 10 DINNER WHITE,CNDL 10 DINNER W,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Active,Rebuyer,No,1237,Y,0.2400,1.2900,N,0.3049,,3,7739,84319,6EA53POZ,TEST RITE - ZEST CANDLE LIMITE,Y,48,12,A,,,
290357,11823,1588,122,62,5,1,-1,-1,S,2003-01-16 00:00:00,N,2016-03-22 00:00:00,10337456,3440,3440 PREMIUM COLORING,10,010 KIDS BOOKS,76,76 BOOKS,6,BOOKS & MAGAZINES,5,ART SUPPLIES & CRAFT,1,Michaels Stores Inc.,,,,,,,,,10337456 BK-EASY CROSSWORD PUZZLES,BK-EASY CROSSWORD,Staple Sku,,CA,Case,2015-06-27 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,N,0.8500,0.0100,Y,0.9265,2003-07-27 00:00:00,3,1453,20616,PP41800,MACPHERSON'S,Y,48,48,,,,
799165,22764,2218,104,26,1,1,-1,-1,S,2013-12-17 00:00:00,Y,2016-05-25 00:00:00,10338719,6758,6758 SUBCLASS NOT IN USE,421,421 JEWELRY COLLECTIONS,52,52 JEWELRY,1,JEWELRY & BEADS,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10338719 T - ROUND GRECIAN - AMETHYST S,T - ROUND GRECIAN - ,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,T,Yes,Warehouse,Active,Buyer,No,0,N,1.4000,0.3900,Y,1.6544,2014-06-29 00:00:00,3,8583,96198,SC042,SOLID OAK INC,Y,144,3,N,8,,
289877,13208,1601,96,3,1,1,-1,-1,S,2003-01-08 00:00:00,Y,2016-02-26 00:00:00,10336935,7345,7345 14 INCH KNIT NEEDLES,102,102 YARN ACCESSORIES,40,40 YARN,3,NEEDLE CRAFTS,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10336935 KNIT NDLS 14" SLV SZ 15,KNIT NDLS 14" SL,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,1.1700,0.4900,Y,1.3826,2011-02-06 00:00:00,3,918,179,11114-15,COATS AND CLARK S SALES CORP,Y,144,6,N,7,7,
289397,13339,1616,96,3,1,1,-1,-1,S,2002-12-18 00:00:00,N,2016-02-26 00:00:00,10336406,7479,7479 SOFTEE CHUNKY-BERNAT,106,106 BASIC ACRYLIC/ACRYLI,40,40 YARN,3,NEEDLE CRAFTS,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10336406 CN YRN SOFTEE CHUNKY GEN GREEN,CN YRN SOFTEE CH,Staple Sku,,CA,Case,2015-06-27 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,1.5000,0.0100,Y,1.5242,2004-05-24 00:00:00,2,1020,1863,16103939232,SPINRITE INC (CANADA),Y,6,6,,,,
287477,18109,2065,131,43,2,1,-1,-1,S,2002-11-15 00:00:00,N,2016-04-30 00:00:00,10334216,2275,2275 ROCKETS,40,040 MODELS & HOBBY,58,58 KIDS ACTIVITY/HOBBY,7,KIDS CRAFTS,2,KIDS/TWEENS,1,Michaels Stores Inc.,,,,,,,,,10334216 ROCKET 2178 HI-FLIER,ROCKET 2178 HI-F,Staple Sku,,EA,Each,2016-04-30 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,3.4000,0.0100,Y,3.7060,2011-11-19 00:00:00,1,6295,55023,EST2178,UNITED MODELS L P,Y,1,1,,,,
286517,11890,1584,97,3,1,1,-1,-1,S,2002-10-28 00:00:00,Y,2016-02-26 00:00:00,10333164,3610,3610 MINI STITCHERY KITS,10,010 STITCHERY KITS,41,41 STITCHERY,3,NEEDLE CRAFTS,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10333164 XS MINI KIT PURRFECT,XS MINI KIT PURR,Staple Sku,,EA,Each,4444-04-04 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,0.7500,0.0100,Y,0.8768,2007-05-28 00:00:00,3,1433,20523,3746,NEEDLE MAGIC, INC,Y,1,1,,,,
286037,13978,1828,48,54,4,1,-1,-1,S,2002-10-16 00:00:00,N,2016-08-05 00:00:00,10332664,8747,8747 TD MATTED PRINTS,43,043 SBA PRINTS/POSTERS,34,34 WALL DECOR,14,FRAMES & WALL DECOR,4,FRAMES & DECOR,1,Michaels Stores Inc.,,,,,,,,,10332664 MAT PRNT 16X20 PALM TREE,MAT PRNT 16X20 P,Staple Sku,,EA,Each,2015-06-20 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,6.8100,9.0000,Y,7.9611,2007-07-09 00:00:00,3,1319,20122,MAZ 058,INNOVATIVE ART,Y,1,1,,,,
285557,13050,1881,116,82,4,1,-1,-1,S,2002-10-01 00:00:00,N,2016-05-21 00:00:00,10332128,6980,6980 PALETTES,66,066 PAINTING ACCESSORIES,70,70 PAINTING ART SUPPLIE,16,ART SUPPLIES,4,FINE ART & FRAMES,1,Michaels Stores Inc.,,,,,,,,,10332128 WATER BUCKETS FOLDING 4 COLORS,WATER BUCKETS FO,Staple Sku,,EA,Each,2016-05-21 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,N,2.0500,0.0100,Y,2.2345,2003-05-10 00:00:00,3,1453,20616,MX3800,MACPHERSON'S,Y,1,1,,,,
797245,11824,1588,122,78,1,1,-1,-1,S,2013-12-02 00:00:00,Y,2015-09-21 00:00:00,10334226,3441,3441 ACTIVITY BOOKS,10,010 KIDS BOOKS,76,76 BOOKS,6,BOOKS & MAGAZINES,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10334226 BK RACE AGAINST TIME,BK RACE AGAINST TIME,Staple Sku,,EA,Each,4444-04-04 00:00:00,,,,,,,Basic,Yes,Direct,Active,Buyer,No,0,N,2.3100,3.9900,N,2.7004,,3,8016,86588,9780385371513,BAKER AND TAYLOR INC,Y,1,1,,,,
796765,23340,2383,120,59,13,1,-1,-1,S,2013-11-25 00:00:00,Y,2016-11-10 00:00:00,10331764,7340,7340 HALL DA,504,504 CANDLE COLLECTIONS,74,74 CANDLES,21,CANDLES,10,SEASONAL,1,Michaels Stores Inc.,,,,,,,,,10331764 TEALIGHT HLDR AST SPIDER,TEALIGHT HLDR AST SP,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Seasonal,Yes,Warehouse,Discontinued,Buyer,No,0,N,1.2300,0.4900,Y,1.5658,2014-10-26 00:00:00,3,8290,90165,HY-C14013,TEST RITE - HONGBAOYUAN TECHNO,Y,3,3,,,,
284597,12156,1728,121,68,13,1,-1,-1,S,2002-09-18 00:00:00,N,2017-01-16 00:00:00,10331053,4497,4497 MINI FLAG,18,018 FLAGS,75,75 IMPULSE,11,IMPULSE,10,SEASONAL,1,Michaels Stores Inc.,,,,,,,,,10331053 GARDEN FLAG-WELCOME TO OUR HME,GARDEN FLAG-WELC,Staple Sku,,CA,Case,2015-06-20 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,N,4.0500,0.0100,Y,4.7346,2005-06-10 00:00:00,3,230,1192,09854021,WINCRAFT INC,Y,2,2,,,,
284117,12934,1865,86,51,12,1,-1,-1,S,2002-09-09 00:00:00,N,2017-01-16 00:00:00,10330498,6721,6721 MINI FAVORS,60,060 FAVORS,25,25 WEDDING,10,RIBBON &  WEDDING,6,CELEBRATIONS,1,Michaels Stores Inc.,,,,,,,,,10330498 FVR WED BBL TIE CAL LIL WHT 12,FVR WED BBL TIE,Staple Sku,,CA,Case,2016-05-21 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,1.9000,0.4900,Y,2.2452,2011-11-06 00:00:00,3,257,1215,MRP10866-12,HIRSCHBERG SCHUTZ AND CO,Y,288,6,A,,8,
283637,12840,1876,107,60,2,1,-1,-1,S,2002-09-05 00:00:00,N,2016-05-23 00:00:00,10329971,6491,6491 SUBCLASS NOT IN USE,64,064 EMBELLISHMENTS,56,56 APPAREL CRAFTS,4,APPAREL CRAFTS,2,KIDS/TWEENS,1,Michaels Stores Inc.,,,,,,,,,10329971 AP-LG / SM FLWR W/CLRD CTR ORG,AP-LG / SM FLWR,Staple Sku,,CA,Case,2015-06-20 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,0.9030,0.0100,Y,1.0671,2009-03-14 00:00:00,3,257,1215,AP12069-106,HIRSCHBERG SCHUTZ AND CO,Y,216,3,N,8,8,
283157,18391,1586,116,33,5,1,-1,-1,S,2002-08-30 00:00:00,N,2017-01-19 00:00:00,10329479,2550,2550 ROYAL SOFT GRIP,10,010 CLASS NOT IN USE,70,70 PAINTING ART SUPPLIE,16,ART SUPPLIES,5,ART SUPPLIES & CRAFT,1,Michaels Stores Inc.,,,,,,,,,10329479 BRSH SG WHITE NYLON BRIGHT 6,BRSH SG WHITE NY,Staple Sku,,CA,Case,2015-06-13 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,1.0000,0.0100,Y,1.0900,2009-12-27 00:00:00,3,1453,20616,RYSG4010-6PP,MACPHERSON'S,Y,3,3,,,,
796285,23294,2263,117,81,12,1,-1,-1,S,2013-11-19 00:00:00,Y,2017-01-17 00:00:00,10330266,7294,7294 TABLECLOTHS,723,723 PARTY GOODS ACCESSOR,71,71 PACKAGING & PARTY,23,PACKAGING & PARTY,6,CELEBRATIONS,1,Michaels Stores Inc.,,,,,,,,,10330266 TABLECOVER 84" ROUND RED,TABLECOVER 84" ROUND,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,1233,Y,0.2800,0.4900,Y,0.3309,2016-03-27 00:00:00,3,6089,785,38180,UNIQUE INDUSTRIES INC,Y,48,6,N,7,,
280277,13330,1616,96,3,1,1,-1,-1,S,2002-08-05 00:00:00,Y,2016-02-26 00:00:00,10326347,7470,7470 SIMPLY SOFT-CARON,106,106 BASIC ACRYLIC/ACRYLI,40,40 YARN,3,NEEDLE CRAFTS,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10326347 YRN SIMPLY SOFT RASPBERRY 7OZ,YRN SIMPLY SOFT,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,1.7700,0.3400,Y,2.0916,2010-04-25 00:00:00,3,353,130,H99004-9920,SPINRITE CORP..,Y,64,4,A,,7,
277397,12383,1833,122,62,5,1,-1,-1,S,2002-05-31 00:00:00,N,2016-04-15 00:00:00,10323104,5457,5457 SUBCLASS NOT IN USE,45,045 ART,76,76 BOOKS,6,BOOKS & MAGAZINES,5,ART SUPPLIES & CRAFT,1,Michaels Stores Inc.,,,,,,,,,10323104 BK-ANIMATING LOONEY TUNES WAY,BK-ANIMATING LOO,Staple Sku,,EA,Each,2015-09-26 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,N,8.6700,5.9900,Y,10.1354,2010-06-22 00:00:00,3,8016,86588,9781560103035,BAKER AND TAYLOR INC,Y,1,1,,,,
276917,12481,1784,48,54,4,1,-1,-1,S,2002-05-20 00:00:00,N,2016-08-05 00:00:00,10322550,5785,5785 INACTIVE 1.3,3,003 FRAMING QUICK CODE 3,34,34 WALL DECOR,14,FRAMES & WALL DECOR,4,FRAMES & DECOR,1,Michaels Stores Inc.,,,,,,,,,10322550 POSTER ROLL50 MIN 16.5"X23.25",POSTER ROLL50 MI,Staple Sku,,EA,Each,2015-06-13 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,N,2.0000,0.0100,Y,2.3381,2003-01-11 00:00:00,3,4062,33303,70501,SCANDECOR,Y,1,1,,,,
792925,20670,2221,100,44,3,1,-1,-1,S,2013-10-16 00:00:00,Y,2016-09-01 00:00:00,10322616,4738,4738 CRAFT TAPE-SBA,443,443 TAPES & TRIMS,44,44 STICKERS & EMBELLISH,8,PAPERCRAFTING,3,PAPERCRAFTING,1,Michaels Stores Inc.,,,,,,,,,10322616 GLITTER TAPE LIME 2YD,GLITTER TAPE LIME 2Y,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,0.8200,0.3900,Y,1.0363,2015-01-11 00:00:00,3,7451,81470,GOR-164,JOYFUL LANDS CRAFT CO LTD,Y,576,3,N,2,,
276437,17286,2098,48,54,4,1,-1,-1,S,2002-05-10 00:00:00,N,2016-08-05 00:00:00,10321989,1495,1495 OPENBACKS,23,023 DIY FRAMING,34,34 WALL DECOR,14,FRAMES & WALL DECOR,4,FINE ART & FRAMES,1,Michaels Stores Inc.,,,,,,,,,10321989 FRM-OW 8X10 GREY BARNWOOD,FRM-OW 8X10 GREY,Staple Sku,,CA,Case,2016-04-30 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,2.3000,0.0100,Y,3.1393,2013-06-23 00:00:00,3,5906,54179,617-2-PT-8X10",PINEFIELD INDUSTRIES LTD (IMPO,Y,3,3,N,4,4,
275957,11548,1727,121,68,13,1,-1,-1,S,2002-05-03 00:00:00,N,2017-01-16 00:00:00,10321438,2163,2163 CHRISTMAS CANDY,17,017 CANDY,75,75 IMPULSE,11,IMPULSE,10,SEASONAL,1,Michaels Stores Inc.,,,,,,,,,10321438 CNDY-BX CHC PRM AST 14.1 POT O,CNDY-BX CHC PRM AS,Staple Sku,,CA,Case,2015-05-30 00:00:00,,,,,,,Seasonal,Yes,Direct,Discontinued,Buyer,No,0,N,4.9500,0.0100,Y,5.3955,2003-11-25 00:00:00,1,5589,49931,01503,TRICOR DISTRIBUTORS,Y,6,6,,,,
274997,15257,1942,97,3,1,1,-1,-1,S,2002-04-16 00:00:00,N,2016-02-26 00:00:00,10320384,9999,9999 DISCONTINUED,997,997 DISCONTINUED,41,41 STITCHERY,3,NEEDLE CRAFTS,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10320384 DOORMAT SNOWMAN JOY ACCENT,DOORMAT SNOWMAN,Staple Sku,,CA,Case,2015-05-30 00:00:00,,,,,,,Seasonal,Yes,Direct,Discontinued,Buyer,No,0,N,2.6000,0.0100,Y,3.0395,2002-12-25 00:00:00,3,2317,2287,12115,BACOVA GUILD, LTD,Y,6,6,,,,
274037,15260,1945,100,44,3,1,-1,-1,S,2002-04-02 00:00:00,N,2016-09-01 00:00:00,10319350,9999,9999 DISCONTINUED,997,997 DISCONTINUED,44,44 STICKERS & EMBELLISH,8,PAPERCRAFTING,3,PAPERCRAFTING,1,Michaels Stores Inc.,,,,,,,,,10319350 3D DECOR MED-BDD PURSE LIME,3D DECOR MED-BDD,Staple Sku,,CA,Case,2016-05-28 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,N,0.9500,0.0100,Y,1.1106,2003-03-29 00:00:00,3,257,1215,TCP0586-56,HIRSCHBERG SCHUTZ AND CO,Y,4,4,,,,
273557,20403,2270,99,44,3,1,-1,-1,S,2002-03-30 00:00:00,N,2016-02-26 00:00:00,10318805,4493,4493 VALUE PK CARDS & ENV,761,761 SOCIAL STATIONARY,43,43 PAPER, CARDS & ALBUM,8,SCRAPBOOKING,3,PAPERCRAFTING,1,Michaels Stores Inc.,,,,,,,,,10318805 CARD/ENV PACK-MED PINK,CARD/ENV PACK-ME,Staple Sku,,CA,Case,2015-05-30 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,0.4500,0.0100,Y,0.5261,2004-05-03 00:00:00,3,257,1215,TCP0250-23,HIRSCHBERG SCHUTZ AND CO,Y,3,3,,,,
273077,15260,1945,100,44,3,1,-1,-1,S,2002-03-27 00:00:00,N,2016-09-01 00:00:00,10318303,9999,9999 DISCONTINUED,997,997 DISCONTINUED,44,44 STICKERS & EMBELLISH,8,SCRAPBOOKING,3,PAPERCRAFTING,1,Michaels Stores Inc.,,,,,,,,,10318303 3D DECOR MED PARTY HORN MULTI,3D DECOR MED PAR,Staple Sku,,CA,Case,2015-05-30 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,0.9500,0.0100,Y,1.1106,2004-11-01 00:00:00,3,257,1215,TCP0016-123,HIRSCHBERG SCHUTZ AND CO,Y,4,4,,,,
272597,26698,1607,85,51,12,1,-1,-1,S,2002-03-21 00:00:00,Y,2017-01-16 00:00:00,10317789,711,0711 $4.99 CMAS RIBBON,104,104 FASHION RIBBON,24,24 RIBBON,10,RIBBON &  WEDDING,6,CELEBRATIONS,1,Michaels Stores Inc.,,,,,,,,,10317789 CORD 3/4Y WIRED PEARL AST,CORD 3/4Y WIRED PEAR,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Seasonal,Yes,Direct,Discontinued,Buyer,No,0,N,0.8500,0.0100,Y,0.9937,2003-11-25 00:00:00,3,257,1215,879,HIRSCHBERG SCHUTZ AND CO,Y,40,40,,,,
271637,15247,1932,85,51,8,1,-1,-1,S,2002-03-08 00:00:00,N,2017-01-16 00:00:00,10316725,9999,9999 DISCONTINUED,997,997 DISCONTINUED,24,24 RIBBON,10,RIBBON &  WEDDING,7,FLORAL/WEDDING,1,Michaels Stores Inc.,,,,,,,,,10316725 RBN OF PINECONE/R PLAID#40 6Y,RBN OF PINECONE/,Staple Sku,,CA,Case,2015-05-30 00:00:00,,,,,,,Seasonal,Yes,Direct,Discontinued,Buyer,No,0,N,1.5200,7.9900,N,1.7769,,3,4295,345,IC6632MC,BERWICK INDUSTRIES  OFFRAY INC,Y,30,30,,,,
271157,26381,1912,105,77,12,1,-1,-1,S,2002-02-28 00:00:00,Y,2016-06-07 00:00:00,10316161,407,0407 FALL/THANKSGIVING,84,084 BAKING TOOLS/ACCESSO,54,54 FOOD CRAFTING,12,FOOD CRAFT,6,CELEBRATIONS,1,Michaels Stores Inc.,,,,,,,,,10316161 FD - SPRINKLES 6 CELL,FD - SPRINKLES 6 CEL,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Seasonal,Yes,Warehouse,Discontinued,Buyer,No,0,N,2.9900,0.6900,Y,3.5333,2016-02-11 00:00:00,3,1338,20158,710-751,WILTON INDUSTRIES (DOMESTIC),Y,48,6,,,,
270197,18425,2168,133,44,3,1,-1,-1,S,2002-02-15 00:00:00,Y,2016-06-09 00:00:00,10315066,2584,2584 OPEN STOCK PENS,682,682 PAPERCRAFTING WRITIN,18,18 MEMORY KEEPING,8,PAPERCRAFTING,3,PAPERCRAFTING,1,Michaels Stores Inc.,,,,,,,,,10315066 GELLYZ GRIP BURGUNDY,GELLYZ GRIP BURG,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,0.7000,0.0100,Y,0.7878,2011-03-21 00:00:00,3,398,135,69029,NEWELL/SANFORD/BEROL,Y,144,12,N,9,9,
789085,11470,1751,49,54,4,1,-1,-1,S,2013-08-28 00:00:00,Y,2016-02-21 00:00:00,10314625,1715,1715 TRENDSETTER,22,022 POSTER FRAMES,35,35 WALL FRAMES,14,FRAMES & WALL DECOR,4,FINE ART & FRAMES,1,Michaels Stores Inc.,,,,,,,,,10314625 FRM 24X36 PEWTER TRENDSETTER,FRM 24X36 PEWTER TRE,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Inactive,Buyer,No,1233,Y,7.5000,18.0000,Y,8.8627,2016-02-21 00:00:00,3,20,101,69413,MCS INDUSTRIES (DOMESTIC),Y,4,4,A,,,
269237,17694,2118,116,82,4,1,-1,-1,S,2002-01-31 00:00:00,Y,2016-03-23 00:00:00,10314015,1894,1894 FLOOR EASELS,321,321 EASELS & FURNITURE,70,70 PAINTING ART SUPPLIE,16,ART SUPPLIES,4,FINE ART & FRAMES,1,Michaels Stores Inc.,,,,,,,,,10314015 EASEL FEILD TRIPOD LARGE,EASEL FEILD TRIP,Staple Sku,,EA,Each,4444-04-04 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,19.5300,4.9900,Y,21.2877,2013-06-23 00:00:00,3,1453,20616,MBM-29,MACPHERSON'S,Y,1,1,,,,
268277,20405,2270,99,44,3,1,-1,-1,S,2002-01-23 00:00:00,N,2016-05-06 00:00:00,10312932,4495,4495 COLOR IT,761,761 SOCIAL STATIONARY,43,43 PAPER, CARDS & ALBUM,8,SCRAPBOOKING,3,PAPERCRAFTING,1,Michaels Stores Inc.,,,,,,,,,10312932 CARD EVERYDAY,CARD EVERYDAY,Staple Sku,,EA,Each,2015-05-23 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,N,1.1950,0.0100,Y,1.3970,2004-06-15 00:00:00,1,470,1461,02005,GIBSON GREETINGS,Y,1,1,,,,
788125,12784,1897,99,44,3,1,-1,-1,S,2013-08-19 00:00:00,Y,2016-07-10 00:00:00,10312865,6389,6389 12X12 PREMIUM PAPER ,79,079 PAPER PADS,43,43 PAPER, CARDS & ALBUM,8,PAPERCRAFTING,3,PAPERCRAFTING,1,Michaels Stores Inc.,,,,,,,,,10312865 PPR PAD 12X12 SOLID BURLAP,PPR PAD 12X12 SOLID ,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,5.8800,7.9900,Y,6.9484,2016-07-10 00:00:00,3,8029,86808,PS-005-00340,DCWV (GS),Y,12,2,N,2,,
267317,11968,1791,106,43,2,1,-1,-1,S,2002-01-18 00:00:00,N,2016-11-02 00:00:00,10311916,3914,3914 FIMO,30,030 KIDS ART MATERIALS,55,55 KIDS ART & EDUCATION,7,KIDS CRAFTS,2,KIDS/TWEENS,1,Michaels Stores Inc.,,,,,,,,,10311916 MODEL MAGIC CHARACTER CREATOR,MODEL MAGIC CHAR,Staple Sku,,CA,Case,2015-05-23 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,10.2700,0.0100,Y,10.5230,2003-12-21 00:00:00,2,1053,1887,04-4848,CRAYOLA CANADA,Y,4,4,,,,
266837,13979,1828,48,54,4,1,-1,-1,S,2002-01-11 00:00:00,N,2016-08-05 00:00:00,10311416,8748,8748 TD FLAT PRINTS,43,043 SBA PRINTS/POSTERS,34,34 WALL DECOR,14,FRAMES & WALL DECOR,4,FRAMES & DECOR,1,Michaels Stores Inc.,,,,,,,,,10311416 PRT 20X28 GIRVAN GARDEN POND,PRT 20X28 GIRVAN,Staple Sku,,EA,Each,2015-05-23 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,5.9100,7.0000,Y,6.9089,2007-07-09 00:00:00,3,1319,20122,GCS 622,INNOVATIVE ART,Y,1,1,,,,
266357,19605,1625,85,51,8,1,-1,-1,S,2002-01-09 00:00:00,N,2017-01-16 00:00:00,10310931,3737,3737 CELEBRATE IT 360,109,109 DECOR RIBBON,24,24 RIBBON,10,RIBBON &  WEDDING,7,FLORAL/WEDDING,1,Michaels Stores Inc.,,,,,,,,,10310931 RBN-EE3Y LT PNK BSKT WV #9,RBN-EE3Y LT PNK,Staple Sku,,CA,Case,2015-05-23 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,1.3400,1.0000,Y,1.5665,2005-11-14 00:00:00,3,4295,345,158237ME,BERWICK INDUSTRIES  OFFRAY INC,Y,3,3,,,,
787645,22559,2368,50,54,4,1,-1,-1,S,2013-08-09 00:00:00,Y,2016-04-26 00:00:00,10311364,6557,6557 LINCOLN PARK,302,302 TABLETOP COLLECTIONS,36,36 TABLE TOP FRAMES,14,FRAMES & WALL DECOR,4,FINE ART & FRAMES,1,Michaels Stores Inc.,,,,,,,,,10311364 FRM 5X7 BLK TORTOISE,FRM 5X7 BLK TORTOISE,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,3.4800,6.0000,Y,4.3352,2014-11-23 00:00:00,3,8245,89471,EX-1202-636 A,LI & FUNG - EXMART INTERNATION,Y,12,3,N,4,,
265877,4662,304,129,23,6,1,-1,-1,S,2002-01-07 00:00:00,Y,2012-10-22 00:00:00,10310447,9090,9090 ALL DEPARTMENTS,40,040 LABELS/FORMS,90,90 SUPPLIES,98,NON-MERCHANDISING,9,NON-MERCHANDISING,1,Michaels Stores Inc.,,,,,,,,,10310447 OVER/SHORT/DAMAGE REPORT,OVER/SHORT/DAMAG,Staple Sku,,EA,Each,4444-04-04 00:00:00,,,,,,,Supplies,No,Direct,Discontinued,Buyer,No,0,N,0.0000,0.0000,N,0.0000,,3,5142,43050,MIC 1506,STAPLES PRINT SOLUTIONS,N,1,1,,,,
265397,20073,2256,119,64,13,1,-1,-1,S,2001-12-28 00:00:00,N,2017-01-16 00:00:00,10309961,4174,4174 MLB,683,683 LICENSED SPORTS,73,73 REGIONAL MERCHANDISI,24,REGIONAL MERCHANDISI,10,SEASONAL,1,Michaels Stores Inc.,,,,,,,,,10309961 CAR FLAG CLEVELAND INDIANS,CAR FLAG CLEVELA,Staple Sku,,CA,Case,2015-05-23 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,N,5.1000,0.0100,Y,5.9620,2005-06-20 00:00:00,3,230,1192,06728011,WINCRAFT INC,Y,2,2,,,,
264917,20067,2256,119,64,13,1,-1,-1,S,2001-12-10 00:00:00,N,2017-01-16 00:00:00,10309432,4168,4168 COLLEGE FOOTBALL,683,683 LICENSED SPORTS,73,73 REGIONAL MERCHANDISI,24,REGIONAL MERCHANDISI,10,SEASONAL,1,Michaels Stores Inc.,,,,,,,,,10309432 CAR FLAG MICHIGAN,CAR FLAG MICHIGA,Staple Sku,,CA,Case,2015-05-23 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,N,5.1000,0.0100,Y,5.9620,2005-06-20 00:00:00,3,230,1192,10544911,WINCRAFT INC,Y,2,2,,,,
262997,11839,1794,122,78,1,1,-1,-1,S,2001-11-12 00:00:00,Y,2015-09-21 00:00:00,10307324,3467,3467 HOME DECORATING,30,030 HOUSE & HOME,76,76 BOOKS,6,BOOKS & MAGAZINES,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10307324 BK - WOOD SHELVES,BK - WOOD SHELVE,Staple Sku,,EA,Each,4444-04-04 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,N,5.2100,0.0100,Y,6.0906,2004-01-30 00:00:00,3,7134,75523,UP09678,HDA INC,Y,1,1,,,,
261796,17714,2123,103,43,2,1,-1,-1,S,2001-10-23 00:00:00,N,2016-03-12 00:00:00,10305978,1909,1909 DOLLS,362,362 SPECIALTY CRAFTS,51,51 KIDS CRAFTS,7,KIDS CRAFTS,2,KIDS/TWEENS,1,Michaels Stores Inc.,,,,,,,,,10305978 MUSIC BOX SMALL WORLD,MUSIC BOX SMALL,Staple Sku,,CA,Case,2016-03-12 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,0.4900,0.0100,Y,0.5790,2006-02-13 00:00:00,3,4571,362,3187-22,FIBRE-CRAFT LLC,Y,84,6,A,,,
785004,14962,1756,81,86,13,1,-1,-1,S,2013-07-05 00:00:00,Y,2016-07-07 00:00:00,10306671,9470,9470 PROVENCE,230,230 STEMS,20,20 FLORAL STEMS/BUSHES,17,FLORAL,10,SEASONAL,1,Michaels Stores Inc.,,,,,,,,,10306671 CN STEM PEONY CORAL,CN STEM PEONY CORAL,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Seasonal,Yes,Warehouse,Active,Buyer,No,0,N,0.5700,4.9900,Y,0.8388,2014-06-15 00:00:00,2,554,1533,YB13G19947CN,DESIGNS FOR ALL SEASONS,Y,120,12,,,,
260836,13053,1881,116,33,5,1,-1,-1,S,2001-10-03 00:00:00,N,2016-02-26 00:00:00,10304944,6983,6983 MRS. LEES,66,066 PAINTING ACCESSORIES,70,70 PAINTING ART SUPPLIE,16,ART SUPPLIES,5,ART SUPPLIES & CRAFT,1,Michaels Stores Inc.,,,,,,,,,10304944 ADHESIVE TEXTURE 118ML,ADHESIVE TEXTURE,Staple Sku,,EA,Each,2015-05-16 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,3.0000,0.5900,Y,3.2700,2007-01-22 00:00:00,3,1453,20616,66-4650,MACPHERSON'S,Y,1,1,,,,
260356,11826,1716,122,78,1,1,-1,-1,S,2001-09-21 00:00:00,Y,2016-04-15 00:00:00,10304411,3445,3445 KNITTING/CROCHET,15,015 TEXTILE CRAFTS,76,76 BOOKS,6,BOOKS & MAGAZINES,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10304411 BK-EASY RUGS TO CROCHET,BK-EASY RUGS TO,Staple Sku,,EA,Each,4444-04-04 00:00:00,,,,,,,Basic,Yes,Direct,Active,Buyer,No,0,N,4.0600,7.0000,N,4.7462,,3,8016,86588,9781574868999,BAKER AND TAYLOR INC,Y,1,1,,,,
784524,22489,2218,104,26,1,1,-1,-1,S,2013-06-28 00:00:00,Y,2016-03-24 00:00:00,10305559,6487,6487 LALIBERI SCARVES AND,421,421 JEWELRY COLLECTIONS,52,52 JEWELRY,1,JEWELRY & BEADS,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10305559 PIN N CLIP BLK RND FEATHER NET,PIN N CLIP BLK RND F,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Seasonal,Yes,Warehouse,Active,Buyer,No,0,N,1.3800,0.4900,N,1.6749,,3,8282,89948,56-63228,EK JEWELRY IDI,Y,144,3,,,,
259396,24439,2147,100,44,3,1,-1,-1,S,2001-08-31 00:00:00,Y,2016-09-01 00:00:00,10303360,8434,8434 BABY-SBA,535,535 FAMILY (STICKERS),44,44 STICKERS & EMBELLISH,8,PAPERCRAFTING,3,PAPERCRAFTING,1,Michaels Stores Inc.,,,,,,,,,10303360 STKR-BABY BOY,STKR-BABY BOY,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,1.0600,0.0100,Y,1.2526,2008-03-24 00:00:00,3,6311,55395,SP-180,ME AND MY BIG IDEAS,Y,288,6,N,4,4,
783084,18634,1701,124,94,13,1,-1,-1,S,2013-06-19 00:00:00,Y,2016-11-18 00:00:00,10302879,2789,2789 TBLTP-COASTAL,135,135 TABLETOP,78,78 HOME ACCENTS,27,HOME ACCENTS,10,SEASONAL,1,Michaels Stores Inc.,,,,,,,,,10302879 T-LIGHT HOLDER METAL BIRDCAGE,T-LIGHT HOLDER METAL,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Seasonal,Yes,Warehouse,Discontinued,Buyer,No,0,N,1.4500,0.6900,Y,1.8488,2014-06-15 00:00:00,3,329,1279,SGM9559,VIGOR INTERNATIONAL H K LIMITE,Y,48,6,,,,
258916,11465,1607,85,27,4,1,-1,-1,S,2001-08-24 00:00:00,N,2017-01-16 00:00:00,10302836,1683,1683 MONOGRAM RIBBON,104,104 FASHION RIBBON,24,24 RIBBON,10,RIBBON &  WEDDING,4,FRAMES & DECOR,1,Michaels Stores Inc.,,,,,,,,,10302836 RBN BUMBLE BEE #40,RBN BUMBLE BEE #,Staple Sku,,EA,Each,2008-05-25 00:00:00,,,,,,,Seasonal,Yes,Direct,Discontinued,Buyer,No,0,N,1.3800,1.5000,Y,1.6133,2002-06-08 00:00:00,3,6179,874,U6969DM #40,HORIZON FABRICS,Y,1,1,,,,
258436,25566,2042,108,66,1,1,-1,-1,S,2001-08-15 00:00:00,Y,2016-04-19 00:00:00,10302324,9558,9558 WOOD BOXES,322,322 BOXES,57,57 UNFINISHED SURFACES,5,UNFINISHED SURFACES,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10302324 UNF WD HINGED BOX 4 AST,UNF WD HINGED BOX 4 ,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Active,Rebuyer,No,1236,Y,0.4000,1.9900,N,0.5240,,3,7558,82589,302324,TEST RITE - CAO XIAN LUYI WOOD,Y,24,24,A,,,
781644,24173,2419,123,58,13,1,-1,-1,D,2013-06-05 00:00:00,Y,2016-02-26 00:00:00,10300649,8178,8178 SPRING 1 DEC BOXES,693,693 DECORATIVE BOXES,77,77 BASKETS & STORAGE,20,ACCENTS & CONTAINERS,10,SEASONAL,1,Michaels Stores Inc.,,,,,,,,,10300649 DP-S/3 BSC TAB TEXTILE BAZAAR,DP-S/3 BSC TAB TEXTILE BAZAAR,Display Pack,,,,4444-04-04 00:00:00,N,,N,,Y,,Seasonal,Yes,Warehouse,Discontinued,Buyer,No,0,N,15.8000,,N,15.8000,,3,6927,69801,3421-60471-MI,TRI COASTAL DESIGN (IMPORT),Y,1,1,,,,
256996,11821,1716,122,78,1,1,-1,-1,S,2001-07-20 00:00:00,Y,2016-04-21 00:00:00,10300678,3436,3436 SEW/STITCH/QUILT,15,015 TEXTILE CRAFTS,76,76 BOOKS,6,BOOKS & MAGAZINES,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10300678 BK-A SNOWMANS FAM ALBUM QT,BK-A SNOWMANS FA,Staple Sku,,EA,Each,4444-04-04 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,N,9.8300,7.0000,Y,11.4915,2005-01-16 00:00:00,3,7134,75523,LA10440,HDA INC,Y,1,1,,,,
256036,12366,1841,107,60,2,1,-1,-1,S,2001-07-03 00:00:00,Y,2016-06-09 00:00:00,10296373,5398,5398 PAINT ACCESSORIES,50,050 PAINTS & DYE,56,56 APPAREL CRAFTS,4,APPAREL CRAFTS,2,KIDS/TWEENS,1,Michaels Stores Inc.,,,,,,,,,10296373 BRUSH CH SHADER,BRUSH CH SHADER,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,0.4400,0.2900,Y,0.5276,2013-03-10 00:00:00,3,7563,82601,10296373,NINGBO CONDA IMP AND EXP CO LT,Y,144,3,N,2,,
255556,11436,1833,122,62,5,1,-1,-1,S,2001-06-26 00:00:00,N,2016-04-15 00:00:00,10294096,1562,1562 SUBCLASS NOT IN USE,45,045 ART,76,76 BOOKS,6,BOOKS & MAGAZINES,5,ART SUPPLIES & CRAFT,1,Michaels Stores Inc.,,,,,,,,,10294096 BK-PTNG SECRET WORLD OF NATURE,BK-PTNG SECRET W,Staple Sku,,EA,Each,2015-05-16 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,N,16.5200,0.0100,Y,18.0068,2003-01-25 00:00:00,3,1453,20616,E031842,MACPHERSON'S,Y,1,1,,,,
251716,20405,2270,99,44,3,1,-1,-1,S,2001-05-02 00:00:00,N,2016-05-06 00:00:00,10183954,4495,4495 COLOR IT,761,761 SOCIAL STATIONARY,43,43 PAPER, CARDS & ALBUM,8,SCRAPBOOKING,3,PAPERCRAFTING,1,Michaels Stores Inc.,,,,,,,,,10183954 GRTG CARD EVERYDY SPD OF LIGHT,GRTG CARD EVERYD,Staple Sku,,CA,Case,2015-04-25 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,N,1.1950,0.0100,Y,1.3026,2004-06-15 00:00:00,1,1160,19687,05713,GIBSON OVERSEAS, INC,Y,4,4,,,,
670043,25244,2453,82,66,1,1,-1,-1,S,2010-12-30 00:00:00,Y,2016-05-13 00:00:00,10185613,9233,9233 OLD MOSAICS,791,791 MOSAICS&STEPPING STO,21,21 HOBBY CRAFTS,5,UNFINISHED SURFACES,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10185613 PEARL SHELLS GREY SQUARES 4 OZ,PEARL SHELLS GREY SQ,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,1.1000,0.4400,Y,1.2806,2013-03-10 00:00:00,3,6819,67088,CSGSMM,AKASHA CRYSTALS  (IMPORT),Y,30,3,N,2,,
251236,15257,1942,97,3,1,1,-1,-1,S,2001-04-27 00:00:00,N,2016-02-26 00:00:00,10165712,9999,9999 DISCONTINUED,997,997 DISCONTINUED,41,41 STITCHERY,3,NEEDLE CRAFTS,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10165712 XS MINI-TO MY DAUGHTER-IN-LAW,XS MINI-TO MY DA,Staple Sku,,EA,Each,2010-08-28 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,3.7000,0.0100,Y,4.3254,2002-03-22 00:00:00,3,1300,20080,03560,JCA, INC,Y,1,1,,,,
250276,15291,1976,97,3,1,1,-1,-1,S,2001-04-23 00:00:00,N,2016-02-26 00:00:00,10138826,9999,9999 SUBCLASS NOT IN USE,998,998 CLASS NOT IN USE,41,41 STITCHERY,3,NEEDLE CRAFTS,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10138826 CHENILLE RIC RAC WHITE,CHENILLE RIC RAC,Staple Sku,,CA,Case,2015-04-18 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,0.9000,0.0100,Y,1.0521,2004-03-15 00:00:00,3,257,1215,SZP 3833-12,HIRSCHBERG SCHUTZ AND CO,Y,3,3,,,,
249796,16557,1837,103,43,2,1,-1,-1,S,2001-04-13 00:00:00,N,2016-02-26 00:00:00,10131557,688,0688 CMAS $1 STICKER BOOK,49,049 SEASONAL,51,51 KIDS CRAFTS,7,KIDS CRAFTS,2,KIDS/TWEENS,1,Michaels Stores Inc.,,,,,,,,,10131557 ORN KIT-ANGEL ABIGAIL,ORN KIT-ANGEL ABIG,Staple Sku,,CA,Case,2015-04-18 00:00:00,,,,,,,Seasonal,Yes,Direct,Discontinued,Buyer,No,0,N,2.4500,0.0100,Y,2.8641,2009-06-26 00:00:00,3,1089,1916,21-81,MAC ENTERPRISES,Y,3,3,,,,
249316,15243,1928,81,34,8,1,-1,-1,S,2001-04-06 00:00:00,N,2015-08-21 00:00:00,10126318,9999,9999 DISCONTINUED,997,997 DISCONTINUED,20,20 FLORAL STEMS/BUSHES,17,FLORAL,7,FLORAL/WEDDING,1,Michaels Stores Inc.,,,,,,,,,10126318 STEM PEONY RED VELVET FABRI,STEM PEONY RED,Staple Sku,,CA,Case,2010-08-31 00:00:00,,,,,,,Seasonal,Yes,Warehouse,Discontinued,Buyer,No,0,N,1.0400,0.7500,Y,1.3842,2002-02-23 00:00:00,3,5462,48070,792962,GRANDE IMPRESSIONS (IMPORT),Y,144,12,,,,
248836,14816,1897,99,44,3,1,-1,-1,S,2001-04-03 00:00:00,N,2016-02-26 00:00:00,10120519,9316,9316 SURFACES,79,079 PAPER PADS,43,43 PAPER, CARDS & ALBUM,8,SCRAPBOOKING,3,PAPERCRAFTING,1,Michaels Stores Inc.,,,,,,,,,10120519 HNDMDE PPR TUBE IND RS PETAL,HNDMDE PPR TUBE IN,Staple Sku,,CA,Case,2015-04-18 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,2.7500,0.0100,Y,3.2148,2005-05-16 00:00:00,3,3288,29497,2001-007,THE PAPER COMPANY,Y,3,3,,,,
724524,13417,1757,83,86,13,1,-1,-1,S,2012-02-06 00:00:00,N,2016-07-23 00:00:00,10234909,7609,7609 CMAS FOLIAGE GARLAND,24,024 FLORAL GARLANDS,22,22 GREENERY,17,FLORAL,10,SEASONAL,1,Michaels Stores Inc.,,,,,,,,,10234909 GRLND CHN HOLLY W/BERRIES A2,GRLND CHN HOLLY W/BE,Staple Sku,,CA,Case,2016-07-23 00:00:00,,,,,,,Seasonal,Yes,Warehouse,Discontinued,Buyer,No,0,N,3.6000,0.0100,Y,5.0889,2014-04-26 00:00:00,3,7798,84652,C-01973#1497,LI & FUNG - ZE YUAN CRAFT FLOW,Y,18,6,,,,
247876,20801,2049,106,43,2,1,-1,-1,S,2001-03-23 00:00:00,N,2016-02-26 00:00:00,10108076,4864,4864 COLLECTIBLES,102,102 SEASONAL,55,55 KIDS ART & EDUCATION,7,KIDS CRAFTS,2,KIDS/TWEENS,1,Michaels Stores Inc.,,,,,,,,,10108076 BODY GLITTER GEL TUBE DISCO,BODY GLITTER GEL,Staple Sku,,CA,Case,2015-04-18 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,N,1.5000,0.0100,Y,1.7535,2003-04-26 00:00:00,3,3521,30680,CFFCGT02,CRAYON FACTORY,Y,6,6,,,,
246436,16667,2062,131,43,2,1,-1,-1,S,2001-03-05 00:00:00,N,2016-02-26 00:00:00,10100547,825,0825 SAND ART,34,034 KIDS ACTIVITIES,58,58 KIDS ACTIVITY/HOBBY,7,KIDS CRAFTS,2,KIDS/TWEENS,1,Michaels Stores Inc.,,,,,,,,,10100547 BOTTLE SAND ART LARGE,BOTTLE SAND ART,Staple Sku,,CA,Case,2010-09-09 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,0.5300,0.0100,Y,0.6263,2004-02-23 00:00:00,1,182,1148,5940,KELLY'S CRAFTS,Y,72,6,A,,,
245956,16664,2062,131,43,2,1,-1,-1,S,2001-02-20 00:00:00,N,2016-03-05 00:00:00,10098132,822,0822 FUZZY POSTER,34,034 KIDS ACTIVITIES,58,58 KIDS ACTIVITY/HOBBY,7,KIDS CRAFTS,2,KIDS/TWEENS,1,Michaels Stores Inc.,,,,,,,,,10098132 POSTER PANORAMA FAIRY LOVE,POSTER PANORAMA,Staple Sku,,CA,Case,2016-03-05 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,2.9100,0.0100,Y,3.4387,2009-12-05 00:00:00,3,642,160,7414,MEGABRANDS AMER INC - ROSEART,Y,12,4,N,2,,
245476,12338,1882,83,34,8,1,-1,-1,S,2001-02-13 00:00:00,N,2015-08-21 00:00:00,10094359,5301,5301 XMAS FRUIT & VEGGIES,67,067 NATURE CENTER,22,22 GREENERY,17,FLORAL,7,FLORAL/WEDDING,1,Michaels Stores Inc.,,,,,,,,,10094359 FRUIT VELVET PEAR/APPL AST,FRUIT VELVET P,Staple Sku,,CA,Case,2008-07-14 00:00:00,,,,,,,Seasonal,Yes,Warehouse,Discontinued,Buyer,No,0,N,0.9900,0.0100,Y,1.2053,2001-11-16 00:00:00,3,4872,3822,EX0444,SILVER RAIN DECORATE CO.,Y,72,12,,,,
243076,15291,1976,97,3,1,1,-1,-1,S,1990-01-01 00:00:00,N,2016-02-26 00:00:00,10808923,9999,9999 SUBCLASS NOT IN USE,998,998 CLASS NOT IN USE,41,41 STITCHERY,3,NEEDLE CRAFTS,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10808923 6" FAB & BATB   ANGEL,6" FAB & BATB ,Staple Sku,,CA,Case,2012-11-24 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,0.9700,0.0100,Y,1.1340,1999-06-01 00:00:00,3,286,1242,CLEARANCE,WIMPOLE STREET CREATIONS,Y,12,12,,,,
242116,11523,1740,118,74,1,1,-1,-1,S,1990-01-01 00:00:00,Y,2016-04-14 00:00:00,10636308,1999,1999 DISC PAINTS,20,020 CRAFT PAINT,72,72 DECORATIVE ART,2,PAINT & DECO ART,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10636308 PAINT CMC BOBBY BLUE PEARL 2OZ,PAINT CMC BOBBY BL,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,0.6700,0.4900,Y,0.7540,2011-05-22 00:00:00,3,895,177,026090202,DELTA CREATIVE INC,Y,36,6,N,4,4,
242596,15293,1978,99,44,3,1,-1,-1,S,1990-01-01 00:00:00,N,2016-02-26 00:00:00,10705210,9999,9999 SUBCLASS NOT IN USE,998,998 CLASS NOT IN USE,43,43 PAPER, CARDS & ALBUM,8,SCRAPBOOKING,3,PAPERCRAFTING,1,Michaels Stores Inc.,,,,,,,,,10705210 PAPER PAINT BLACK,PAPER PAINT BLAC,Staple Sku,,CA,Case,2008-06-20 00:00:00,,,,,,,Seasonal,Yes,Warehouse,Discontinued,Buyer,No,0,N,1.0500,0.0100,Y,1.1817,1996-04-28 00:00:00,3,726,168,CLEARANCE,TULIP,Y,60,6,,,,
241156,22073,1740,118,57,5,1,-1,-1,S,1990-01-01 00:00:00,N,2016-03-21 00:00:00,10604165,6072,6072 SUBCLASS NOT IN USE,20,020 CRAFT PAINT,72,72 DECORATIVE ART,2,PAINT & DECO ART,5,ART SUPPLIES & CRAFT,1,Michaels Stores Inc.,,,,,,,,,10604165 SPEEDBALL PNTR CD VIOLET F,SPEEDBALL PNTR C,Staple Sku,,CA,Case,2013-05-11 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,N,1.1500,0.0100,Y,1.3590,2003-03-15 00:00:00,3,402,138,M7348,ELMER'S PRODUCTS, INC..,Y,144,6,A,,,
239716,13491,1893,118,74,1,1,-1,-1,S,1990-01-01 00:00:00,Y,2016-03-01 00:00:00,10523282,7704,7704 DSN MSTR SPRAY PAINT,77,077 SPRAY PAINT,72,72 DECORATIVE ART,2,PAINT & DECO ART,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10523282 DSN MST PURPLE,DSN MST PURPLE,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Active,Rebuyer,No,1199,Y,2.6200,7.9900,N,3.0960,,3,75,1055,740,DESIGN MASTER COLOR TOOL INC,Y,4,4,N,9,9,
801565,19265,2196,118,74,1,1,-1,-1,S,2013-12-31 00:00:00,Y,2016-04-18 00:00:00,10346909,3410,3410 MARTHA 2 OZ SATIN,61,061 MARTHA PAINT/ACC,72,72 DECORATIVE ART,2,PAINT & DECO ART,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10346909 MS 2OZ STN-DATE NIGHT PINK,MS 2OZ STN-DATE NIGH,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Active,Rebuyer,No,1241,Y,0.9500,2.2900,N,1.1226,,3,5307,455,33485,PLAID ENTERPRISES INC,Y,24,3,N,7,,
787165,25505,2037,108,66,1,1,-1,-1,S,2013-08-05 00:00:00,Y,2017-01-19 00:00:00,10309522,9497,9497 TOOLS,312,312 TOOLS,57,57 UNFINISHED SURFACES,5,UNFINISHED SURFACES,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10309522 X-O CARVING TOOL SET TRILING C,X-O CARVING TOOL SET,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Active,Rebuyer,No,1239,Y,16.2500,41.9900,N,19.2026,,3,402,138,X5179T,ELMER S PRODUCTS INC,Y,12,1,N,7,,
798685,27208,1792,107,60,2,1,-1,-1,S,2013-12-11 00:00:00,Y,2016-06-16 00:00:00,10337757,1218,1218 BABY WEARABLES,30,030 WEARABLE ACCESSORIES,56,56 APPAREL CRAFTS,4,APPAREL CRAFTS,2,KIDS/TWEENS,1,Michaels Stores Inc.,,,,,,,,,10337757 COWBOY BOOT 6-9 MONTH,COWBOY BOOT 6-9 MONT,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Basic,Yes,Warehouse,Discontinued,Buyer,No,0,Y,3.3000,2.4900,Y,3.8996,2015-10-11 00:00:00,3,7140,75559,GNBB3671MS2,ABG ACCESSORIES,Y,48,3,N,4,,
781164,25933,2493,81,86,13,1,-1,-1,S,2013-06-02 00:00:00,Y,2017-02-08 00:00:00,10300190,9943,9943 SIG ACCENTS BUNDLE,832,832 BUNDLES,20,20 FLORAL STEMS/BUSHES,17,FLORAL,10,SEASONAL,1,Michaels Stores Inc.,,,,,,,,,10300190 STEM BDL RSE HYD LP/PK/WH A3,STEM BDL RSE HYD LP/,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Seasonal,Yes,Warehouse,Discontinued,Buyer,No,0,N,1.6100,0.9900,Y,2.1138,2014-06-15 00:00:00,3,694,1650,MSI-533800457ABC,GLOBAL BEST INDUSTRIAL LTD,Y,120,6,,,,
779724,24860,1668,92,56,13,1,-1,-1,S,2013-05-21 00:00:00,Y,2016-06-15 00:00:00,10298354,8848,8848 VAL WALL,12,012 DECOR,30,30 SPRING HOLIDAY,19,MINI SEASON,10,SEASONAL,1,Michaels Stores Inc.,,,,,,,,,10298354 ORN 6" BEADED HEART 2A,ORN 6" BEADED HEART ,Staple Sku,,CA,Case,4444-04-04 00:00:00,,,,,,,Seasonal,Yes,Warehouse,Discontinued,Buyer,No,0,N,0.8800,0.0100,Y,1.1564,2014-06-18 00:00:00,3,50,10347,12BK60AS00100,UCP INTERNATIONAL CO LTD (IMPO,Y,6,6,,,,
234916,20153,2241,118,57,5,1,-1,-1,S,1990-01-01 00:00:00,N,2016-04-18 00:00:00,10297959,4249,4249 SUBCLASS NOT IN USE,647,647 STENCILS/WALL DECOR,72,72 DECORATIVE ART,2,PAINT & DECO ART,5,ART SUPPLIES & CRAFT,1,Michaels Stores Inc.,,,,,,,,,10297959 BRASS STEN CSUAL SCRIPT,BRASS STEN CSUAL,Staple Sku,,CA,Case,2015-05-16 00:00:00,,,,,,,Basic,Yes,Direct,Discontinued,Buyer,No,0,Y,2.8000,0.0100,Y,3.2733,2004-02-14 00:00:00,3,2018,2161,GS142,MOMENTA INC,Y,3,3,,,,
231556,20748,2284,105,53,1,1,-1,-1,S,1990-01-01 00:00:00,N,2016-04-20 00:00:00,10293590,4813,4813 CMAS BAKING TOOLS,981,981 FASHION,54,54 FOOD CRAFTING,12,FOOD CRAFT,1,GENERAL CRAFTING,1,Michaels Stores Inc.,,,,,,,,,10293590 MOLD CMAS BULBS LOLLI,MOLD CMAS BULBS,Staple Sku,,CA,Case,2015-05-16 00:00:00,,,,,,,Seasonal,Yes,Direct,Discontinued,Buyer,No,0,N,1.0600,0.0100,Y,1.2392,2010-02-27 00:00:00,3,1338,20158,2115-1721,WILTON INDUSTRIES (DOMESTIC),Y,6,6,,,,"""

In [47]:
sizes = set()
for line in samp.split('\n'):
    sizes.add(len(line.split(',')))
sizes

{72, 73, 74}